In [1]:
import pandas as pd
import numpy as np

In [2]:
### Get all the pillar names from the excel

In [3]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [4]:
col_names = ['Indicator','check', 'Data Source','Data Link','Raw/Index','Filename','Sub-Pillar']

In [5]:
names = names[col_names]

In [6]:
names.head()

,Indicator,check,Data Source,Data Link,Raw/Index,Filename,Sub-Pillar
0,Countries,NaN,UN Statistics Division: List of Countries,https://unstats.un.org,Raw,Countries,NaN
1,"Database of Global Administrative Areas (GADM,...",NaN,NaN,https://gadm.org,Raw,NaN,NaN
2,High Resolution Population Density Maps + Demo...,NaN,NaN,NaN,NaN,NaN,NaN
3,population density vs openstreetmap object den...,NaN,NaN,NaN,NaN,NaN,NaN
4,Population Density,Infrastructure,World Bank: World Development Indicators,https://datacatalog.worldbank.org,Raw,population_density,Connectivity Technology


In [7]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [8]:
data_stats

,Filename,Indicator
check,,
Business,19,28
Foundations,13,22
Government,11,16
Infrastructure,47,58
People,35,49
Regulation,5,8
Strategy,1,1


In [9]:
### People

In [10]:
bnames = names[(names.check=='People')&(~names.Filename.isna())]#&(names.Index==False)]
bnames

,Indicator,check,Data Source,Data Link,Raw/Index,Filename,Sub-Pillar
115,Human Capital Index (HCI),People,UN: E-Government Survey,https://publicadministration.un.org/egovkb,Index,e_government_index,Digital Literacy Skills
116,% of population using internet (all),People,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,Index,ITU_database,Digital Literacy Skills
117,% of population using internet (female),People,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,Index,ITU_database,Digital Literacy Skills
118,% of population using internet (male),People,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,Index,ITU_database,Digital Literacy Skills
119,% of internet users who own cryptocurrency,People,Statista,https://www.statista.com,Index,cryptocurrency_adoption,Digital Literacy Skills
120,Global Crypto Adoption Index,People,Chainanalysis: Global Crypto Adoption Index,https://go.chainalysis.com,Index,Chainalysis_2021_Geography_Cryptocurrency_Report,Usage and ownership
121,SDG 4.4 Digital literacy data,People,UNESCO: Sustainable Development Goals,http://data.uis.unesco.org,Raw,SDG_digital_literacy_data,Digital Literacy Skills
122,UNDP Human Development Index (HDI),People,UNDP: Human Development Data Center,http://hdr.undp.org/en,Index,undp_human_developmnt,Culture
123,Facebook Social Connectedness Index,People,Facebook: Social Connectedness Index,https://data.humdata.org,NaN,fb_social_connectedness,Culture
124,Share of individuals using the Internet to int...,People,OECD: Going Digital Toolkit,http://goingdigital.oecd.org,Index,population_interacting_public_officials,Culture


In [11]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()
subpillars = bnames['Sub-Pillar'].unique()

In [12]:
# get all file names
bfiles = bnames.Filename.unique()

In [13]:
bfiles

array(['e_government_index', 'ITU_database', 'cryptocurrency_adoption',
       'Chainalysis_2021_Geography_Cryptocurrency_Report',
       'SDG_digital_literacy_data', 'undp_human_developmnt',
       'fb_social_connectedness',
       'population_interacting_public_officials',
       'digital_public_service_use', 'apps_in_national_language',
       'time_spent_online', 'happiness_score',
       'not_buying_online_concern_about_returning',
       'not_buying_online_concern_about_security',
       'ewaste_per_inhabitant', 'automation_led_unemployment',
       'cyberbullying_rate', 'global_wellbeing_initiative',
       'financial_inclusiveness', 'individuals_buying_online_frequency',
       'social_media_penetration', 'FB_users', 'gender_gaps',
       'population_digital_financial_services',
       'tax_percent_mobile_ownership', 'population_with_smartphones'],
      dtype=object)

In [14]:
subpillars

array(['Digital Literacy Skills', 'Usage and ownership', 'Culture',
       'Digital Wellbeing'], dtype=object)

In [15]:
# formula for converting scale 0-100
def convert_rank(old_value, old_min, old_max, new_min=1, new_max=5.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [16]:
# formula for converting scale 0-1
def convert_rank_b(old_value, old_min=0, old_max=1, new_min=1, new_max=5.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [17]:
# formula for converting scale 0-10
def convert_rank_c(old_value, old_min=0, old_max=10, new_min=1, new_max=5.99 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

In [18]:
### Add population dataset
df = pd.read_csv('../../processed/Population.csv')

# Normalize country names as much as possible
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
df['Country Name'] = df['Country Name'].replace('Bahrain (Kingdom of)','Bahrain')
df['Country Name'] = df['Country Name'].replace('Bolivia','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Bolivia, Plurinational State of','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Brunei','Brunei Darussalam')
df['Country Name'] = df['Country Name'].replace('Bulgaria (Rep.)','Bulgaria')
df['Country Name'] = df['Country Name'].replace('Central African Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Central African Rep.','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace("China (People's Rep.)",'China')
df['Country Name'] = df['Country Name'].replace("Comoros",'Comoros (the)')
df['Country Name'] = df['Country Name'].replace("Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Brazzaville)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Rep. of the)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Democratic Republic of the)",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, Dem. Rep.",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, The Democratic Republic of the",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("DR Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Dem. Rep. of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Cote d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Côte d’Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote D'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote dIvoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cōte d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Ivory Coast","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Dem. People's Rep. of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Democratic People's Republic of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Korea, Dem. People's Rep.","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("North Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hong Kong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hongkong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].replace("Eswatini (Kingdom of)",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Swaziland",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Faröe Islands",'Faroe Islands')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Gambia.*$)', 'Gambia (the)')
df['Country Name'] = df['Country Name'].replace("Georgia (Country)",'Georgia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Iran.*$)', 'Iran (Islamic Republic of)')
df['Country Name'] = df['Country Name'].replace("Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep. of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep.)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Republic of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, Rep.*$)', 'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea, South",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("South Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Republic of Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kyrgyz.*$)', 'Kyrgyzstan')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lao.*$)', "Lao People's Democratic Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macao.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macau.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Micronesia.*$)', "Micronesia (Federated States of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Moldova.*$)', "Republic of Moldova (the)")
df['Country Name'] = df['Country Name'].replace("Morroco",'Morocco')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Nepal.*$)', "Nepal")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].replace("Niger",'Niger (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macedonia.*$)', "North Macedonia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Palestin.*$)', "State of Palestine (the)")
df['Country Name'] = df['Country Name'].replace("West Bank and Gaza",'State of Palestine (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Panama.*$)', "Panama")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Philippines.*$)', "Philippines (the)")
df['Country Name'] = df['Country Name'].replace("Republic of the Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Myanmar.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Puerto Rico.*$)', "Puerto Rico")
df['Country Name'] = df['Country Name'].replace("Russia",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].replace("Russian Federation",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Slovak.*$)', "Slovakia")
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan (the)\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*South Sudan.*$)', "South Sudan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Syria.*$)', "Syrian Arab Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*São Tomé.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taiwan.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taipei.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tanzania.*$)', "United Republic of Tanzania (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Netherlands.*$)', "Netherlands (the)")
df['Country Name'] = df['Country Name'].replace("UAE",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("U.A.E",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("United Arab Emirates",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace('United Kingdom','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('UK','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("Great Britain",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("United Kingdom of Great Britain and Northern Ireland",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('Vietnam','Viet Nam')
df['Country Name'] = df['Country Name'].replace('United States','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('USA','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('United States of America','United States of America (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Virgin Islands.*$)', "United States Virgin Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vatican.*$)', "Vatican")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Venezuela.*$)', "Venezuela (Bolivarian Republic of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Yemen.*$)', "Yemen")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Arab world.*$)', "Arab World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*World.*$)', "World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kitts and Nevis.*$)', "Saint Kitts and Nevis")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lucia.*$)', "Saint Lucia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Martin (French Part).*$)', "Saint Martin (French Part)")
df['Country Name'] = df['Country Name'].replace('Sint Maarten','Saint Martin')
df['Country Name'] = df['Country Name'].replace('St. Martin (French part)','Saint Martin (French Part)')
df['Country Name'] = df['Country Name'].replace('Sint Maarten (Dutch part)','Saint Martin (Dutch Part)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent and the Grenadines.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Verde.*$)', "Cabo Verde")
df['Country Name'] = df['Country Name'].replace('Congo, Democratic Republic','Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo, Rep.','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo (Rep.)','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Egypt.*$)', "Egypt")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, D.*$)', "Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tobago.*$)', "Trinidad and Tobago")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Timor-Leste.*$)', "Timor-Leste")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Emirates.*$)', "United Arab Emirates (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Papua.*$)', "Papua New Guinea")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bissau.*$)', "Guinea-Bissau")
df['Country Name'] = df['Country Name'].replace('Eq. Guinea','Equatorial Guinea')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Burma.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].replace('C.A. Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Ant.& Barb.','Antigua and Barbuda')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bosnia.*$)', "Bosnia and Herzegovina")
df['Country Name'] = df['Country Name'].replace('Domin. Rep.','Dominican Republic (the)')
df['Country Name'] = df['Country Name'].replace('Dominica (Commonwealth of)','Dominica')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*European Union.*$)', "European Union")
df['Country Name'] = df['Country Name'].replace('R. of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Principe.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Solomon.*$)', "Solomon Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].replace('Curacao','Curaçao')
df['Country Name'] = df['Country Name'].replace('Reunion','Réunion')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kosovo.*$)', "Kosovo (UNSCR 1244)")

pop = df
pop

<ipython-input-18-35a075b1c6b0>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
<ipython-input-18-35a075b1c6b0>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
<ipython-input-18-35a075b1c6b0>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
<ipython-input-18-35a075b1c6b0>:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
<ipython-input-18-35a075b1c6b0>:38: FutureWarning: The default value of regex will change from True to 

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2015,2016,2017,2018,2019,2020,Country Name,Country Code,Indicator Name,Indicator Code
0,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,57702.0,58044.0,58377.0,58734.0,...,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0,Aruba,ABW,"Population, total",SP.POP.TOTL
1,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,152752671.0,156876454.0,161156430.0,165611760.0,...,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL
2,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,10174840.0,10399936.0,10637064.0,10893772.0,...,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0,Afghanistan,AFG,"Population, total",SP.POP.TOTL
3,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,109701811.0,112195950.0,114781116.0,117468741.0,...,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL
4,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,5781305.0,5774440.0,5771973.0,5803677.0,...,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0,Angola,AGO,"Population, total",SP.POP.TOTL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,1106000.0,1135000.0,1163000.0,1191000.0,...,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1775378.0,Kosovo (UNSCR 1244),XKX,"Population, total",SP.POP.TOTL
262,5315351.0,5393034.0,5473671.0,5556767.0,5641598.0,5727745.0,5816241.0,5907873.0,6001858.0,6097042.0,...,26497881.0,27168210.0,27834811.0,28498683.0,29161922.0,29825968.0,Yemen,YEM,"Population, total",SP.POP.TOTL
263,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,19888259.0,20406863.0,20942147.0,21496075.0,...,55386369.0,56207649.0,57009751.0,57792520.0,58558267.0,59308690.0,South Africa,ZAF,"Population, total",SP.POP.TOTL
264,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,3681953.0,3797877.0,3918872.0,4045740.0,...,15879370.0,16363449.0,16853608.0,17351714.0,17861034.0,18383956.0,Zambia,ZMB,"Population, total",SP.POP.TOTL


In [19]:
### 1. Human Capital Index (HCI)

In [20]:
indicators[0]

# load data
indicator = indicators[0]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Human Capital Index (HCI)
e_government_index


In [21]:
df.head(10)

,Survey Year,Country Name,E-Government Rank,E-Government Index,E-Participation Index,Online Service Index,Human Capital Index,Telecommunication Infrastructure Index
0,2020,Iraq,143,0.4360,0.3095,0.3353,0.4358,0.5370
1,2020,Ireland,27,0.8433,0.8571,0.7706,0.9494,0.8100
2,2020,Israel,30,0.8361,0.7143,0.7471,0.8924,0.8689
3,2020,Italy,37,0.8231,0.8214,0.8294,0.8466,0.7932
4,2020,Jamaica,114,0.5392,0.3690,0.3882,0.7142,0.5151
5,2020,Japan,14,0.8989,0.9881,0.9059,0.8684,0.9223
6,2020,Jordan,117,0.5309,0.3333,0.3588,0.6800,0.5540
7,2020,Kazakhstan,29,0.8375,0.8810,0.9235,0.8866,0.7024
8,2020,Kenya,116,0.5326,0.5952,0.6765,0.5812,0.3402
9,2020,Kiribati,145,0.4320,0.5595,0.4941,0.6778,0.1241


In [22]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [23]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Human Capital Index'] 
df['Year'] = df['Survey Year']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))

In [24]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Iraq,2020,Human Capital Index (HCI),0.4358,3.174642,True,Digital Literacy Skills
1,Ireland,2020,Human Capital Index (HCI),0.9494,5.737506,True,Digital Literacy Skills
2,Israel,2020,Human Capital Index (HCI),0.8924,5.453076,True,Digital Literacy Skills
3,Italy,2020,Human Capital Index (HCI),0.8466,5.224534,True,Digital Literacy Skills
4,Jamaica,2020,Human Capital Index (HCI),0.7142,4.563858,True,Digital Literacy Skills
...,...,...,...,...,...,...,...
188,Senegal,2020,Human Capital Index (HCI),0.3332,2.662668,True,Digital Literacy Skills
189,Serbia,2020,Human Capital Index (HCI),0.8280,5.131720,True,Digital Literacy Skills
190,Seychelles,2020,Human Capital Index (HCI),0.7660,4.822340,True,Digital Literacy Skills
191,Singapore,2020,Human Capital Index (HCI),0.8904,5.443096,True,Digital Literacy Skills


In [25]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [26]:
### 2. % of population using internet (all)

In [27]:
indicators[1]

# load data
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using internet (all)
ITU_database


In [28]:
df.head(150)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
145,North Macedonia,Europe,MKD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
146,Norway,Europe,NOR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
147,Poland,Europe,POL,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
148,Portugal,Europe,PRT,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [29]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [30]:
df = df[(df.Year==2019)]
df = df[(df.iloc[:,3] =='Individuals using the Internet, total (%)')]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [31]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
47607,Cabo Verde,2019.0,% of population using internet (all),61.943398,4.090976,True,Digital Literacy Skills
47608,Central African Rep.,2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
47609,Congo (Rep. of the),2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
47610,Côte d'Ivoire,2019.0,% of population using internet (all),36.288955,2.810819,True,Digital Literacy Skills
47611,Equatorial Guinea,2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
...,...,...,...,...,...,...,...
47739,Saint Vincent and the Grenadines,2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
47740,Trinidad and Tobago,2019.0,% of population using internet (all),NaN,NaN,True,Digital Literacy Skills
47741,United States,2019.0,% of population using internet (all),89.430285,5.462571,True,Digital Literacy Skills
47742,Uruguay,2019.0,% of population using internet (all),83.351534,5.159242,True,Digital Literacy Skills


In [32]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [33]:
### 3. % of population using internet (female)

In [34]:
indicators[2]

# load data
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using internet (female)
ITU_database


In [35]:
df.head(10)

# Must convert the string in the dataset to float

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [36]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [37]:
df = df[(df.iloc[:,3] =='Female Internet users as a % of total female population')]
df = df[(df.Year==2019)]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [38]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
40307,Cabo Verde,2019.0,% of population using internet (female),60.965017,4.042154,True,Digital Literacy Skills
40308,Côte d'Ivoire,2019.0,% of population using internet (female),32.924445,2.642930,True,Digital Literacy Skills
40309,Kenya,2019.0,% of population using internet (female),20.125122,2.004244,True,Digital Literacy Skills
40310,Lesotho,2019.0,% of population using internet (female),44.922768,3.241646,True,Digital Literacy Skills
40311,Mauritius,2019.0,% of population using internet (female),60.130645,4.000519,True,Digital Literacy Skills
...,...,...,...,...,...,...,...
40388,Mexico,2019.0,% of population using internet (female),68.574653,4.421875,True,Digital Literacy Skills
40389,Panama,2019.0,% of population using internet (female),63.811494,4.184194,True,Digital Literacy Skills
40390,Paraguay,2019.0,% of population using internet (female),69.017230,4.443960,True,Digital Literacy Skills
40391,Peru,2019.0,% of population using internet (female),56.992144,3.843908,True,Digital Literacy Skills


In [39]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [40]:
### 4. % of population using internet (male)

In [41]:
indicators[3]

# load data
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using internet (male)
ITU_database


In [42]:
df.head(10)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [43]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [44]:
df = df[(df.iloc[:,3] =='Male Internet users as a % of total male population')]
df = df[(df.Year==2019)]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [45]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
41078,Cabo Verde,2019.0,% of population using internet (male),62.921166,4.139766,True,Digital Literacy Skills
41079,Côte d'Ivoire,2019.0,% of population using internet (male),39.808871,2.986463,True,Digital Literacy Skills
41080,Kenya,2019.0,% of population using internet (male),25.071668,2.251076,True,Digital Literacy Skills
41081,Lesotho,2019.0,% of population using internet (male),38.349985,2.913664,True,Digital Literacy Skills
41082,Mauritius,2019.0,% of population using internet (male),63.393093,4.163315,True,Digital Literacy Skills
...,...,...,...,...,...,...,...
41159,Mexico,2019.0,% of population using internet (male),71.740324,4.579842,True,Digital Literacy Skills
41160,Panama,2019.0,% of population using internet (male),63.433835,4.165348,True,Digital Literacy Skills
41161,Paraguay,2019.0,% of population using internet (male),68.006259,4.393512,True,Digital Literacy Skills
41162,Peru,2019.0,% of population using internet (male),62.915703,4.139494,True,Digital Literacy Skills


In [46]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [47]:
### 5. % of internet users who own cryptocurrency

In [48]:
indicators[4]

# load data
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# The indicators are all jumbled, need to unclutter this

% of internet users who own cryptocurrency
cryptocurrency_adoption


In [49]:
df.head(15)

,Country,Share of respondents who indicated they either owned or used cryptocurrencies in 55 countries worldwide in 2020,Units
0,Nigeria,31.9,in %
1,Vietnam,21.1,in %
2,Philippines,19.8,in %
3,South Africa,17.8,in %
4,Thailand,17.6,in %
5,Peru,16.1,in %
6,Turkey,16.1,in %
7,Colombia,15.3,in %
8,Argentina,14.4,in %
9,Indonesia,13.0,in %


In [50]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [51]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Share of respondents who indicated they either owned or used cryptocurrencies in 55 countries worldwide in 2020'] 
df['Country Name'] = df['Country']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [52]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Nigeria,2020,% of internet users who own cryptocurrency,31.9,2.59181,True,Digital Literacy Skills
1,Vietnam,2020,% of internet users who own cryptocurrency,21.1,2.05289,True,Digital Literacy Skills
2,Philippines,2020,% of internet users who own cryptocurrency,19.8,1.98802,True,Digital Literacy Skills
3,South Africa,2020,% of internet users who own cryptocurrency,17.8,1.88822,True,Digital Literacy Skills
4,Thailand,2020,% of internet users who own cryptocurrency,17.6,1.87824,True,Digital Literacy Skills
5,Peru,2020,% of internet users who own cryptocurrency,16.1,1.80339,True,Digital Literacy Skills
6,Turkey,2020,% of internet users who own cryptocurrency,16.1,1.80339,True,Digital Literacy Skills
7,Colombia,2020,% of internet users who own cryptocurrency,15.3,1.76347,True,Digital Literacy Skills
8,Argentina,2020,% of internet users who own cryptocurrency,14.4,1.71856,True,Digital Literacy Skills
9,Indonesia,2020,% of internet users who own cryptocurrency,13.0,1.64870,True,Digital Literacy Skills


In [53]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [54]:
### 6. Global Crypto Adoption Index

In [55]:
indicators[5]

# load data
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Global Crypto Adoption Index
Chainalysis_2021_Geography_Cryptocurrency_Report


In [56]:
df.head(15)

,Country,Score,Rank,On-chain value received (Rank),On-chain retail value received (Rank),P2P exchange trade volume (Rank)
0,Vietnam,1.00,1,4,2,3
1,India,0.37,2,2,3,72
2,Pakistan,0.36,3,11,12,8
3,Ukraine,0.29,4,6,5,40
4,Kenya,0.28,5,41,28,1
5,Nigeria,0.26,6,15,10,18
6,Venezuela,0.25,7,29,22,6
7,United States,0.22,8,3,4,109
8,Togo,0.19,9,47,42,2
9,Argentina,0.19,10,14,17,33


In [57]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [58]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Score'] 
df['Country Name'] = df['Country']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))

In [59]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Vietnam,2019,Global Crypto Adoption Index,1.00,5.9900,True,Usage and ownership
1,India,2019,Global Crypto Adoption Index,0.37,2.8463,True,Usage and ownership
2,Pakistan,2019,Global Crypto Adoption Index,0.36,2.7964,True,Usage and ownership
3,Ukraine,2019,Global Crypto Adoption Index,0.29,2.4471,True,Usage and ownership
4,Kenya,2019,Global Crypto Adoption Index,0.28,2.3972,True,Usage and ownership
...,...,...,...,...,...,...,...
152,Guyana,2019,Global Crypto Adoption Index,0.00,1.0000,True,Usage and ownership
153,"Virgin Islands, U.S.",2019,Global Crypto Adoption Index,0.00,1.0000,True,Usage and ownership
154,Brunei Darussalam,2019,Global Crypto Adoption Index,0.00,1.0000,True,Usage and ownership
155,Bermuda,2019,Global Crypto Adoption Index,0.00,1.0000,True,Usage and ownership


In [60]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [61]:
### 7. SDG 4.4 Digital literacy data

In [62]:
indicators[6]

# load data
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))


SDG 4.4 Digital literacy data
SDG_digital_literacy_data


In [63]:
df.head(15)

,SDG_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,ICTSKILLTRANSFERFILE_M,Proportion of youth and adults who have transf...,BRA,Brazil,2014,2014,23.31007,NaN,NaN
1,ICTSKILLTRANSFERFILE_M,Proportion of youth and adults who have transf...,BRA,Brazil,2016,2016,21.53173,NaN,NaN
2,ICTSKILLTRANSFERFILE_M,Proportion of youth and adults who have transf...,BRA,Brazil,2017,2017,21.14880,NaN,NaN
3,ICTSKILLTRANSFERFILE_M,Proportion of youth and adults who have transf...,BRA,Brazil,2018,2018,21.84886,NaN,NaN
4,ICTSKILLDUPLIC_M,Proportion of youth and adults who have used c...,BRA,Brazil,2014,2014,25.64427,NaN,NaN
5,ICTSKILLDUPLIC_M,Proportion of youth and adults who have used c...,BRA,Brazil,2016,2016,22.01463,NaN,NaN
6,ICTSKILLDUPLIC_M,Proportion of youth and adults who have used c...,BRA,Brazil,2017,2017,22.69577,NaN,NaN
7,ICTSKILLDUPLIC_M,Proportion of youth and adults who have used c...,BRA,Brazil,2018,2018,22.59234,NaN,NaN
8,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,TUR,Turkey,2014,2014,27.72807,NaN,NaN
9,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,TUR,Turkey,2015,2015,25.66142,NaN,NaN


In [64]:
df = df[(df.Time == 2019)]
df['Country Name'] = df['Country']
df.head(15)

,SDG_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags,Country Name
12,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,TUR,Turkey,2019,2019,38.18440,NaN,NaN,Turkey
35,ICTSKILLSOFTWARE,"Proportion of youth and adults who have found,...",THA,Thailand,2019,2019,5.30000,NaN,NaN,Thailand
75,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,PAK,Pakistan,2019,2019,2.70000,NaN,NaN,Pakistan
90,ICTSKILLPROGLANG,Proportion of youth and adults who have wrote ...,PRT,Portugal,2019,2019,8.20000,NaN,NaN,Portugal
103,ICTSKILLSOFTWARE,"Proportion of youth and adults who have found,...",FIN,Finland,2019,2019,63.10000,NaN,NaN,Finland
121,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,FRA,France,2019,2019,54.60000,NaN,NaN,France
126,ICTSKILLTRANSFERFILE,Proportion of youth and adults who have transf...,ESP,Spain,2019,2019,54.67742,NaN,NaN,Spain
133,ICTSKILLATTACH,Proportion of youth and adults who have sent e...,KAZ,Kazakhstan,2019,2019,51.70000,NaN,NaN,Kazakhstan
143,ICTSKILLPROGLANG,Proportion of youth and adults who have wrote ...,KAZ,Kazakhstan,2019,2019,6.30000,NaN,NaN,Kazakhstan
152,ICTSKILLPROGLANG,Proportion of youth and adults who have wrote ...,BEL,Belgium,2019,2019,4.50000,NaN,NaN,Belgium


In [65]:
subpillars[0]
subpillar = subpillars[0]
print(subpillar)

Digital Literacy Skills


In [66]:
# Since there can be several entries for the same country, calculate the mean of the value for each country
agg_df = df.groupby(['Country Name']).agg({'Value':'mean'})
agg_df.columns = ['Mean' ]
agg_df

,Mean
Country Name,
Albania,11.528571
Austria,48.320000
Bahrain,47.700000
Bangladesh,0.600000
Belarus,26.811111
...,...
Thailand,11.944444
Tunisia,18.888889
Turkey,28.284636


In [67]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
agg_df['higher_is_better'] = True
agg_df['Indicator'] = indicator
agg_df['data_col'] = agg_df['Mean']
agg_df['Year'] = 2019
agg_df['Sub-Pillar'] = subpillar

min_rank = agg_df['data_col'].min()
max_rank = agg_df['data_col'].max()

# transform 0-1 rank into 1-6
agg_df['new_rank_score'] = agg_df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [68]:
agg_df = agg_df[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
agg_df

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
Albania,2019,SDG 4.4 Digital literacy data,11.528571,1.936084,True,Digital Literacy Skills
Austria,2019,SDG 4.4 Digital literacy data,48.320000,5.087444,True,Digital Literacy Skills
Bahrain,2019,SDG 4.4 Digital literacy data,47.700000,5.034338,True,Digital Literacy Skills
Bangladesh,2019,SDG 4.4 Digital literacy data,0.600000,1.000000,True,Digital Literacy Skills
Belarus,2019,SDG 4.4 Digital literacy data,26.811111,3.245106,True,Digital Literacy Skills
...,...,...,...,...,...,...
Thailand,2019,SDG 4.4 Digital literacy data,11.944444,1.971705,True,Digital Literacy Skills
Tunisia,2019,SDG 4.4 Digital literacy data,18.888889,2.566530,True,Digital Literacy Skills
Turkey,2019,SDG 4.4 Digital literacy data,28.284636,3.371320,True,Digital Literacy Skills


In [69]:
### 8. UNDP Human Development Index (HDI)

In [70]:
indicators[7]

# load data
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

UNDP Human Development Index (HDI) 
undp_human_developmnt


In [71]:
df.head()

,HDI rank,Country,Value,Unnamed: 3,(years),(2017 PPP $),data_country,data_year
0,NaN,NaN,2019,NaN,2019,2019,NaN,NaN
1,NaN,VERY HIGH HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Norway,0.957,NaN,12.89775,66494.25217,NaN,NaN
3,2.0,Ireland,0.955,NaN,12.6663305,68370.58737,NaN,NaN
4,2.0,Switzerland,0.955,NaN,13.38081241,69393.52076,NaN,NaN


In [72]:
# choose only the rows where column HDI rank is numeric

df = df.iloc[0:194,:]
df = df[pd.to_numeric(df['HDI rank'], errors='coerce').notnull()]

In [73]:
df.head(67)

,HDI rank,Country,Value,Unnamed: 3,(years),(2017 PPP $),data_country,data_year
2,1.0,Norway,0.957,NaN,12.89775,66494.25217,NaN,NaN
3,2.0,Ireland,0.955,NaN,12.6663305,68370.58737,NaN,NaN
4,2.0,Switzerland,0.955,NaN,13.38081241,69393.52076,NaN,NaN
5,4.0,"Hong Kong, China (SAR)",0.949,NaN,12.27996,62984.76553,NaN,NaN
6,4.0,Iceland,0.949,NaN,12.77278684,54682.38057,NaN,NaN
...,...,...,...,...,...,...,...,...
64,62.0,Malaysia,0.81,NaN,10.37283,27534.09856,NaN,NaN
65,64.0,Kuwait,0.806,NaN,7.275667996,58590.08219,NaN,NaN
66,64.0,Serbia,0.806,NaN,11.19411,17191.66873,NaN,NaN
67,66.0,Mauritius,0.804,NaN,9.54,25266.21195,NaN,NaN


In [74]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [75]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'].astype(float)
df['Country Name'] = df['Country']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))

In [76]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Norway,2019,UNDP Human Development Index (HDI),0.957,5.77543,True,Culture
3,Ireland,2019,UNDP Human Development Index (HDI),0.955,5.76545,True,Culture
4,Switzerland,2019,UNDP Human Development Index (HDI),0.955,5.76545,True,Culture
5,"Hong Kong, China (SAR)",2019,UNDP Human Development Index (HDI),0.949,5.73551,True,Culture
6,Iceland,2019,UNDP Human Development Index (HDI),0.949,5.73551,True,Culture
...,...,...,...,...,...,...,...
189,Burundi,2019,UNDP Human Development Index (HDI),0.433,3.16067,True,Culture
190,South Sudan,2019,UNDP Human Development Index (HDI),0.433,3.16067,True,Culture
191,Chad,2019,UNDP Human Development Index (HDI),0.398,2.98602,True,Culture
192,Central African Republic,2019,UNDP Human Development Index (HDI),0.397,2.98103,True,Culture


In [77]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator, index=False))

In [78]:
### 9. Facebook Social Connectedness Index

In [79]:
indicators[8]

# load data
indicator = indicators[8]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Facebook Social Connectedness Index
fb_social_connectedness


In [80]:
df.head(25)

,user_loc,fr_loc,scaled_sci
0,1001,AE,8729
1,1001,AG,95256
2,1001,AL,3122
3,1001,AM,3470
4,1001,AO,2839
5,1001,AR,3729
6,1001,AT,6977
7,1001,AU,21136
8,1001,AW,27607
9,1001,AZ,1108


In [81]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [82]:
df['Country Name'] = df['fr_loc']

In [83]:
agg_df = df.groupby(['Country Name']).agg({'scaled_sci':'mean'})
agg_df

,scaled_sci
Country Name,
AE,10370.571075
AG,184463.177454
AL,8414.715701
AM,2825.352741
AO,3777.043357
...,...
XK,9482.109322
YT,1927.744503
ZA,9507.912666


In [84]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
agg_df['higher_is_better'] = True
agg_df['Indicator'] = indicator
agg_df['data_col'] = agg_df['scaled_sci'] 
agg_df['Sub-Pillar'] = subpillar
agg_df['Year'] = 2021

min_rank = agg_df['data_col'].min()
max_rank = agg_df['data_col'].max()

# transform 0-1 rank into 1-6
agg_df['new_rank_score'] = agg_df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

In [85]:
agg_df = agg_df[['Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
agg_df

# Need to find a way to convert ISO codes to full country names

,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
Country Name,,,,,,
AE,2021,Facebook Social Connectedness Index,10370.571075,1.011276,True,Culture
AG,2021,Facebook Social Connectedness Index,184463.177454,1.231168,True,Culture
AL,2021,Facebook Social Connectedness Index,8414.715701,1.008805,True,Culture
AM,2021,Facebook Social Connectedness Index,2825.352741,1.001745,True,Culture
AO,2021,Facebook Social Connectedness Index,3777.043357,1.002947,True,Culture
...,...,...,...,...,...,...
XK,2021,Facebook Social Connectedness Index,9482.109322,1.010153,True,Culture
YT,2021,Facebook Social Connectedness Index,1927.744503,1.000612,True,Culture
ZA,2021,Facebook Social Connectedness Index,9507.912666,1.010186,True,Culture


In [86]:
### 10. Share of individuals using the Internet to interact with officials

In [87]:
indicators[9]

# load data
indicator = indicators[9]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Share of individuals using the Internet to interact with public authorities
population_interacting_public_officials


In [88]:
df.head(15)

,Country,Indicator,Breakdowns,Time,Unit,PowerCode,Reference Period,Value,Flags
0,Australia,Individuals using the Internet for downloading...,All (individuals aged 16-74),2010,Percentage,Units,NaN,38.1100,Difference in methodology
1,Australia,Individuals using the Internet for downloading...,All (individuals aged 16-74),2012,Percentage,Units,NaN,49.9600,Difference in methodology
2,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2005,Percentage,Units,NaN,29.1940,NaN
3,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2006,Percentage,Units,NaN,32.9733,NaN
4,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2007,Percentage,Units,NaN,27.4741,NaN
5,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2008,Percentage,Units,NaN,51.2252,Break
6,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2009,Percentage,Units,NaN,48.8515,NaN
7,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2010,Percentage,Units,NaN,51.0458,NaN
8,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2011,Percentage,Units,NaN,51.2893,NaN
9,Austria,Individuals using the Internet for visiting or...,All (individuals aged 16-74),2012,Percentage,Units,NaN,52.8274,NaN


In [89]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [90]:
# filter most recent year
df = df[(df.Time==2019)]
df = df[(df.Indicator=='Individuals using the Internet for visiting or interacting with public authorities websites - last 12 m (%)')]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Year'] = df['Time']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [91]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
16,Austria,2019,Share of individuals using the Internet to int...,69.735500,4.479801,True,Culture
64,Belgium,2019,Share of individuals using the Internet to int...,58.605000,3.924389,True,Culture
109,Brazil,2019,Share of individuals using the Internet to int...,34.194129,2.706287,True,Culture
148,Colombia,2019,Share of individuals using the Internet to int...,6.274581,1.313102,True,Culture
169,Czech Republic,2019,Share of individuals using the Internet to int...,53.776600,3.683452,True,Culture
216,Denmark,2019,Share of individuals using the Internet to int...,91.670900,5.574378,True,Culture
262,Estonia,2019,Share of individuals using the Internet to int...,80.015600,4.992778,True,Culture
310,Finland,2019,Share of individuals using the Internet to int...,87.295400,5.356040,True,Culture
357,France,2019,Share of individuals using the Internet to int...,74.724200,4.728738,True,Culture
397,Germany,2019,Share of individuals using the Internet to int...,59.097700,3.948975,True,Culture


In [92]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [93]:
### 11. Level of satisfaction for online public service

In [94]:
indicators[10]

# load data
indicator = indicators[10]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Level of satisfaction for online public services (% of users, by type of interaction and service)
digital_public_service_use


In [95]:
df.head(10)

,Country Name,User Satisfaction (%)
0,United Arab Emirates,88.5
1,India,86.1
2,Saudi Arabia,84.2
3,China,81.8
4,Singapore,80.9
5,France,79.1
6,Chile,78.3
7,Hong Kong,76.6
8,Kenya,76.1
9,Poland,75.2


In [96]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [97]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['User Satisfaction (%)'] 
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# Need to replace the % mark 

In [98]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,United Arab Emirates,2020,Level of satisfaction for online public servic...,88.5,5.41615,True,Culture
1,India,2020,Level of satisfaction for online public servic...,86.1,5.29639,True,Culture
2,Saudi Arabia,2020,Level of satisfaction for online public servic...,84.2,5.20158,True,Culture
3,China,2020,Level of satisfaction for online public servic...,81.8,5.08182,True,Culture
4,Singapore,2020,Level of satisfaction for online public servic...,80.9,5.03691,True,Culture
5,France,2020,Level of satisfaction for online public servic...,79.1,4.94709,True,Culture
6,Chile,2020,Level of satisfaction for online public servic...,78.3,4.90717,True,Culture
7,Hong Kong,2020,Level of satisfaction for online public servic...,76.6,4.82234,True,Culture
8,Kenya,2020,Level of satisfaction for online public servic...,76.1,4.79739,True,Culture
9,Poland,2020,Level of satisfaction for online public servic...,75.2,4.75248,True,Culture


In [99]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [100]:
### 12. Number of mobile apps available in national language

In [101]:
indicators[11]

# load data
indicator = indicators[11]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Number of mobile apps available in national language(s)
apps_in_national_language


In [102]:
df.head(15)

,Unnamed: 0,ISO Code,Country,Region,Year,Number of apps in national language
0,1,AFG,Afghanistan,South Asia,2014,2.444741
1,2,AFG,Afghanistan,South Asia,2015,2.793221
2,3,AFG,Afghanistan,South Asia,2016,2.849881
3,4,AFG,Afghanistan,South Asia,2017,2.913741
4,5,AFG,Afghanistan,South Asia,2018,2.961247
5,6,AFG,Afghanistan,South Asia,2019,3.000000
6,7,AGO,Angola,Sub-Saharan Africa,2014,53.333237
7,8,AGO,Angola,Sub-Saharan Africa,2015,55.080910
8,9,AGO,Angola,Sub-Saharan Africa,2016,56.516411
9,10,AGO,Angola,Sub-Saharan Africa,2017,57.061077


In [103]:
subpillars[2]
subpillar = subpillars[2]
print(subpillar)

Culture


In [104]:
# filter most recent year and global value
df = df[(df.Year==2019)]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Number of apps in national language'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [105]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Number of mobile apps available in national la...,3.000000,1.149700,True,Culture
11,Angola,2019,Number of mobile apps available in national la...,57.978500,3.893127,True,Culture
17,Albania,2019,Number of mobile apps available in national la...,67.286446,4.357594,True,Culture
23,United Arab Emirates,2019,Number of mobile apps available in national la...,78.453438,4.914827,True,Culture
29,Argentina,2019,Number of mobile apps available in national la...,89.724289,5.477242,True,Culture
...,...,...,...,...,...,...,...
995,Samoa,2019,Number of mobile apps available in national la...,58.999996,3.944100,True,Culture
1001,Yemen,2019,Number of mobile apps available in national la...,78.453438,4.914827,True,Culture
1007,South Africa,2019,Number of mobile apps available in national la...,57.271473,3.857846,True,Culture
1013,Zambia,2019,Number of mobile apps available in national la...,16.000000,1.798400,True,Culture


In [106]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [107]:
### 13. Device Addiction (time of use on internet)

In [108]:
indicators[12]

# load data
indicator = indicators[12]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Device Addiction (time of use on internet / on devices)
time_spent_online


In [109]:
df.head(15)

# Need to move the row further down

,Country,"Average daily time spent using the internet by online users worldwide as of 3rd quarter 2020, by region (in hours.minutes)"
0,Philippines,10.56
1,Brazil,10.08
2,Colombia,10.07
3,South Africa,10.06
4,Argentina,9.39
5,Malaysia,9.17
6,Mexico,9.01
7,Indonesia,8.52
8,Thailand,8.44
9,Taiwan,8.08


In [110]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital Wellbeing


In [111]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Average daily time spent using the internet by online users worldwide as of 3rd quarter 2020, by region (in hours.minutes)'] 
df['Country Name'] = df['Country']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

In [112]:
first = df['data_col'].quantile(1/5)
second = df['data_col'].quantile(2/5)
third = df['data_col'].quantile(3/5)
fourth = df['data_col'].quantile(4/5)
fifth = df['data_col'].quantile(5/5)
data = {'Min':min_rank,'Max':max_rank,'1':first,'2':second,'3':third,'4':fourth,'5':fifth}
quintile = pd.DataFrame(data, index=[0])
quintile

,Min,Max,1,2,3,4,5
0,4.25,10.56,5.424,6.348,7.244,8.296,10.56


In [113]:
# Convert the results into quintile
df['new_rank_score'] = pd.qcut(df['data_col'], 5, labels=[1,2,3,4,5])

df=df[df['new_rank_score'].notna()]

# Convert the results into quintile
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: convert_rank(row, old_min=1,old_max=5))
# need to invert score since higher rank is not better 
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)
df

,Country,"Average daily time spent using the internet by online users worldwide as of 3rd quarter 2020, by region (in hours.minutes)",higher_is_better,Indicator,data_col,Country Name,Year,Sub-Pillar,new_rank_score
0,Philippines,10.56,False,Device Addiction (time of use on internet / on...,10.56,Philippines,2020,Digital Wellbeing,1.0000
1,Brazil,10.08,False,Device Addiction (time of use on internet / on...,10.08,Brazil,2020,Digital Wellbeing,1.0000
2,Colombia,10.07,False,Device Addiction (time of use on internet / on...,10.07,Colombia,2020,Digital Wellbeing,1.0000
3,South Africa,10.06,False,Device Addiction (time of use on internet / on...,10.06,South Africa,2020,Digital Wellbeing,1.0000
4,Argentina,9.39,False,Device Addiction (time of use on internet / on...,9.39,Argentina,2020,Digital Wellbeing,1.0000
5,Malaysia,9.17,False,Device Addiction (time of use on internet / on...,9.17,Malaysia,2020,Digital Wellbeing,1.0000
6,Mexico,9.01,False,Device Addiction (time of use on internet / on...,9.01,Mexico,2020,Digital Wellbeing,1.0000
7,Indonesia,8.52,False,Device Addiction (time of use on internet / on...,8.52,Indonesia,2020,Digital Wellbeing,1.0000
8,Thailand,8.44,False,Device Addiction (time of use on internet / on...,8.44,Thailand,2020,Digital Wellbeing,1.0000
9,Taiwan,8.08,False,Device Addiction (time of use on internet / on...,8.08,Taiwan,2020,Digital Wellbeing,2.2475


In [114]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Philippines,2020,Device Addiction (time of use on internet / on...,10.56,1.0000,False,Digital Wellbeing
1,Brazil,2020,Device Addiction (time of use on internet / on...,10.08,1.0000,False,Digital Wellbeing
2,Colombia,2020,Device Addiction (time of use on internet / on...,10.07,1.0000,False,Digital Wellbeing
3,South Africa,2020,Device Addiction (time of use on internet / on...,10.06,1.0000,False,Digital Wellbeing
4,Argentina,2020,Device Addiction (time of use on internet / on...,9.39,1.0000,False,Digital Wellbeing
5,Malaysia,2020,Device Addiction (time of use on internet / on...,9.17,1.0000,False,Digital Wellbeing
6,Mexico,2020,Device Addiction (time of use on internet / on...,9.01,1.0000,False,Digital Wellbeing
7,Indonesia,2020,Device Addiction (time of use on internet / on...,8.52,1.0000,False,Digital Wellbeing
8,Thailand,2020,Device Addiction (time of use on internet / on...,8.44,1.0000,False,Digital Wellbeing
9,Taiwan,2020,Device Addiction (time of use on internet / on...,8.08,2.2475,False,Digital Wellbeing


In [115]:
#df.to_csv('../indicator_scores/people_{}_scores.csv'.format(bf), index=False)
df.insert(0,'Pillar','People')
df.to_csv('../non-index/people_{}_scores.csv'.format(bf), index=False)

In [116]:
### 14. Gross National Wellbeing

In [117]:
indicators[13]

# load data
indicator = indicators[13]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# happiness_score is missing looks like the problem is in a redundant space before it in the filename matching

Gross National Wellbeing
happiness_score


In [118]:
df.head(15)

,RANK,COUNTRY/ECONOMY,VALUE,SCORE
0,1.0,Finland,7.78,100.00
1,2.0,Switzerland,7.69,98.31
2,3.0,Denmark,7.69,98.28
3,4.0,Iceland,7.53,95.13
4,5.0,Norway,7.44,93.35
5,6.0,Netherlands,7.43,93.02
6,7.0,Luxembourg,7.40,92.60
7,8.0,Sweden,7.40,92.49
8,9.0,Ireland,7.25,89.67
9,10.0,Australia,7.23,89.26


In [119]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital Wellbeing


In [120]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['SCORE'] 
df['Country Name'] = df['COUNTRY/ECONOMY']
df['Year'] = 2019
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [121]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Finland,2019,Gross National Wellbeing,100.00,5.990000,True,Digital Wellbeing
1,Switzerland,2019,Gross National Wellbeing,98.31,5.905669,True,Digital Wellbeing
2,Denmark,2019,Gross National Wellbeing,98.28,5.904172,True,Digital Wellbeing
3,Iceland,2019,Gross National Wellbeing,95.13,5.746987,True,Digital Wellbeing
4,Norway,2019,Gross National Wellbeing,93.35,5.658165,True,Digital Wellbeing
...,...,...,...,...,...,...,...
129,Zambia,2019,Gross National Wellbeing,12.06,1.601794,True,Digital Wellbeing
130,Rwanda,2019,Gross National Wellbeing,11.30,1.563870,True,Digital Wellbeing
131,India,2019,Gross National Wellbeing,10.92,1.544908,True,Digital Wellbeing
132,Zimbabwe,2019,Gross National Wellbeing,0.00,1.000000,True,Digital Wellbeing


In [122]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [123]:
### 15. Percentage of individuals not buying online due to concerns about returning products

In [124]:
indicators[14]

# load data
indicator = indicators[14]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Percentage of individuals not buying online due to concerns about returning products
not_buying_online_concern_about_returning


In [125]:
df.head(15)

,Indicator,Country,Variable,Unit,Scope,Time,Value,Flags
0,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2005,10.358200,NaN
1,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2006,14.697300,NaN
2,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2009,25.081000,NaN
3,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2015,17.049300,NaN
4,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2017,15.114670,NaN
5,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2019,16.727490,NaN
6,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2005,7.911900,NaN
7,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2006,13.751800,NaN
8,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2009,24.068800,NaN
9,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2015,12.671000,NaN


In [126]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital Wellbeing


In [127]:
# filter most recent year and global value
df = df[(df.Time==2019)]
df = df[(df.Scope =='All individuals (aged 16-74)')]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Year'] = df['Time']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)

In [128]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Austria,2019,Percentage of individuals not buying online du...,16.727490,5.155298,False,Digital Wellbeing
22,Belgium,2019,Percentage of individuals not buying online du...,9.164280,5.532702,False,Digital Wellbeing
37,Czech Republic,2019,Percentage of individuals not buying online du...,8.857784,5.547997,False,Digital Wellbeing
51,Denmark,2019,Percentage of individuals not buying online du...,13.273220,5.327666,False,Digital Wellbeing
67,Estonia,2019,Percentage of individuals not buying online du...,3.968984,5.791948,False,Digital Wellbeing
121,European Union (28 countries),2019,Percentage of individuals not buying online du...,17.256510,5.128900,False,Digital Wellbeing
135,Finland,2019,Percentage of individuals not buying online du...,46.960710,3.646661,False,Digital Wellbeing
152,France,2019,Percentage of individuals not buying online du...,14.236370,5.279605,False,Digital Wellbeing
167,Germany,2019,Percentage of individuals not buying online du...,16.450070,5.169142,False,Digital Wellbeing
184,Greece,2019,Percentage of individuals not buying online du...,9.599324,5.510994,False,Digital Wellbeing


In [129]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [130]:
### 16. Percentage of individuals not buying online due to payment security concerns

In [131]:
indicators[15]

# load data
indicator = indicators[15]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Percentage of individuals not buying online due to payment security concerns
not_buying_online_concern_about_security


In [132]:
df.head(15)

,Indicator,Country,Variable,Unit,Scope,Time,Value,Flags
0,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2009,38.80930,NaN
1,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2015,34.89850,NaN
2,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2017,33.04743,NaN
3,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,All individuals (aged 16-74),2019,31.19888,NaN
4,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2009,38.34010,NaN
5,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2015,34.96870,NaN
6,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2017,16.65675,NaN
7,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 16-24,2019,12.88121,NaN
8,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 55-74,2009,40.57610,NaN
9,Individuals who did not buy online for payment...,Austria,"Individuals who, in the last 12 months, haven'...",Percentage of individuals who ordered goods or...,Individuals aged 55-74,2015,42.60200,NaN


In [133]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital Wellbeing


In [134]:
# filter most recent year and global value
df = df[(df.Time==2019)]
df = df[(df.Scope =='All individuals (aged 16-74)')]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Year'] = df['Time']
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)

In [135]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
3,Austria,2019,Percentage of individuals not buying online du...,31.198880,4.433176,False,Digital Wellbeing
15,Belgium,2019,Percentage of individuals not buying online du...,18.129610,5.085332,False,Digital Wellbeing
27,Czech Republic,2019,Percentage of individuals not buying online du...,9.814267,5.500268,False,Digital Wellbeing
39,Denmark,2019,Percentage of individuals not buying online du...,18.770550,5.053350,False,Digital Wellbeing
51,Estonia,2019,Percentage of individuals not buying online du...,4.357608,5.772555,False,Digital Wellbeing
84,European Union (28 countries),2019,Percentage of individuals not buying online du...,24.308600,4.777001,False,Digital Wellbeing
96,Finland,2019,Percentage of individuals not buying online du...,64.427210,2.775082,False,Digital Wellbeing
108,France,2019,Percentage of individuals not buying online du...,40.843490,3.951910,False,Digital Wellbeing
120,Germany,2019,Percentage of individuals not buying online du...,25.121840,4.736420,False,Digital Wellbeing
132,Greece,2019,Percentage of individuals not buying online du...,23.223810,4.831132,False,Digital Wellbeing


In [136]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [137]:
### 17. E-waste generated, kilograms per inhabitant

In [138]:
indicators[16]

# load data
indicator = indicators[16]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

E-waste generated, kilograms per inhabitant
ewaste_per_inhabitant


In [139]:
df.head(15)

,iso3c,region_id,country_name,income_id,gdp,composition_food_organic_waste_percent,composition_glass_percent,composition_metal_percent,composition_other_percent,composition_paper_cardboard_percent,...,waste_treatment_controlled_landfill_percent,waste_treatment_incineration_percent,waste_treatment_landfill_unspecified_percent,waste_treatment_open_dump_percent,waste_treatment_other_percent,waste_treatment_recycling_percent,waste_treatment_sanitary_landfill_landfill_gas_system_percent,waste_treatment_unaccounted_for_percent,waste_treatment_waterways_marine_percent,where_where_is_this_data_measured
0,ABW,LCN,Aruba,HIC,35563.312500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.000,NaN,89.00,NaN,NaN
1,AFG,SAS,Afghanistan,LIC,2057.062256,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
2,AGO,SSF,Angola,LMC,8036.690430,51.80,6.70,4.40,11.50,11.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,ECS,Albania,UMC,13724.058594,51.40,4.50,4.80,15.21,9.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Some disposal sites
4,AND,ECS,Andorra,HIC,43711.800781,31.20,8.20,2.60,11.60,35.10,...,NaN,52.10,NaN,NaN,NaN,NaN,NaN,47.90,NaN,NaN
5,ARE,MEA,United Arab Emirates,HIC,67119.132812,39.00,4.00,3.00,10.00,25.00,...,NaN,NaN,9.00,62.0,NaN,20.000,NaN,NaN,NaN,NaN
6,ARG,LCN,Argentina,HIC,23550.099609,38.74,3.16,1.84,15.36,13.96,...,8.90,NaN,NaN,22.6,NaN,6.000,62.5,NaN,NaN,Other
7,ARM,ECS,Armenia,UMC,11019.838867,57.00,3.20,3.40,17.40,6.70,...,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,Other
8,ASM,EAS,American Samoa,UMC,11113.442383,19.70,3.40,7.90,25.60,26.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ATG,LCN,Antigua and Barbuda,HIC,17965.501953,46.00,7.00,7.00,12.00,15.00,...,98.68,NaN,NaN,NaN,NaN,NaN,NaN,1.14,0.1,Disposal Site


In [140]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital Wellbeing


In [141]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
# Use special_waste_e_waste_tons_year times one thousand and divde by total population
df['data_col'] = df['special_waste_e_waste_tons_year']
df['Year'] = 2020
df['Country Name'] = df['country_name']
df['Sub-Pillar'] = subpillar

In [142]:
df = df[['Country Name', 'Year','Indicator','data_col','higher_is_better','Sub-Pillar']]

# Normalize country names as much as possible
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
df['Country Name'] = df['Country Name'].replace('Bahrain (Kingdom of)','Bahrain')
df['Country Name'] = df['Country Name'].replace('Bolivia','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Bolivia, Plurinational State of','Bolivia (Plurinational State of)')
df['Country Name'] = df['Country Name'].replace('Brunei','Brunei Darussalam')
df['Country Name'] = df['Country Name'].replace('Bulgaria (Rep.)','Bulgaria')
df['Country Name'] = df['Country Name'].replace('Central African Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Central African Rep.','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace("China (People's Rep.)",'China')
df['Country Name'] = df['Country Name'].replace("Comoros",'Comoros (the)')
df['Country Name'] = df['Country Name'].replace("Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Brazzaville)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Rep. of the)",'Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo (Democratic Republic of the)",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, Dem. Rep.",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Congo, The Democratic Republic of the",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("DR Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Democratic Republic of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Dem. Rep. of the Congo",'Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace("Cote d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Côte d’Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote D'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cote dIvoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Cōte d'Ivoire","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Ivory Coast","Côte d'Ivoire")
df['Country Name'] = df['Country Name'].replace("Dem. People's Rep. of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Democratic People's Republic of Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("Korea, Dem. People's Rep.","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].replace("North Korea","Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hong Kong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Hongkong.*$)', 'China, Hong Kong Special Administrative Region')
df['Country Name'] = df['Country Name'].replace("Eswatini (Kingdom of)",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Swaziland",'Eswatini')
df['Country Name'] = df['Country Name'].replace("Faröe Islands",'Faroe Islands')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Gambia.*$)', 'Gambia (the)')
df['Country Name'] = df['Country Name'].replace("Georgia (Country)",'Georgia')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Iran.*$)', 'Iran (Islamic Republic of)')
df['Country Name'] = df['Country Name'].replace("Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep. of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Rep.)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea (Republic of)",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, Rep.*$)', 'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Korea, South",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("South Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace("Republic of Korea",'Republic of Korea (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kyrgyz.*$)', 'Kyrgyzstan')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lao.*$)', "Lao People's Democratic Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macao.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macau.*$)', "China, Macao Special Administrative Region")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Micronesia.*$)', "Micronesia (Federated States of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Moldova.*$)', "Republic of Moldova (the)")
df['Country Name'] = df['Country Name'].replace("Morroco",'Morocco')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Nepal.*$)', "Nepal")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].replace("Niger",'Niger (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Macedonia.*$)', "North Macedonia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*New Ze.*$)', "New Zealand")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Palestin.*$)', "State of Palestine (the)")
df['Country Name'] = df['Country Name'].replace("West Bank and Gaza",'State of Palestine (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Panama.*$)', "Panama")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Philippines.*$)', "Philippines (the)")
df['Country Name'] = df['Country Name'].replace("Republic of the Congo",'Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Myanmar.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Puerto Rico.*$)', "Puerto Rico")
df['Country Name'] = df['Country Name'].replace("Russia",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].replace("Russian Federation",'Russian Federation (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Slovak.*$)', "Slovakia")
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'\bSudan (the)\b', 'Sudan (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*South Sudan.*$)', "South Sudan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Syria.*$)', "Syrian Arab Republic (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*São Tomé.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taiwan.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Taipei.*$)', "Taiwan")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tanzania.*$)', "United Republic of Tanzania (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Netherlands.*$)', "Netherlands (the)")
df['Country Name'] = df['Country Name'].replace("UAE",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("U.A.E",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace("United Arab Emirates",'United Arab Emirates (the)')
df['Country Name'] = df['Country Name'].replace('United Kingdom','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('UK','United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("Great Britain",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace("United Kingdom of Great Britain and Northern Ireland",'United Kingdom of Great Britain and Northern Ireland (the)')
df['Country Name'] = df['Country Name'].replace('Vietnam','Viet Nam')
df['Country Name'] = df['Country Name'].replace('United States','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('USA','United States of America (the)')
df['Country Name'] = df['Country Name'].replace('United States of America','United States of America (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Virgin Islands.*$)', "United States Virgin Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vatican.*$)', "Vatican")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Venezuela.*$)', "Venezuela (Bolivarian Republic of)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Yemen.*$)', "Yemen")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Arab world.*$)', "Arab World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*World.*$)', "World")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kitts and Nevis.*$)', "Saint Kitts and Nevis")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Lucia.*$)', "Saint Lucia")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Martin (French Part).*$)', "Saint Martin (French Part)")
df['Country Name'] = df['Country Name'].replace('Sint Maarten','Saint Martin')
df['Country Name'] = df['Country Name'].replace('St. Martin (French part)','Saint Martin (French Part)')
df['Country Name'] = df['Country Name'].replace('Sint Maarten (Dutch part)','Saint Martin (Dutch Part)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent and the Grenadines.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Verde.*$)', "Cabo Verde")
df['Country Name'] = df['Country Name'].replace('Congo, Democratic Republic','Democratic Republic of the Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo, Rep.','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Republic of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].replace('Congo (Rep.)','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Egypt.*$)', "Egypt")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Korea, D.*$)', "Democratic People's Republic of Korea (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Tobago.*$)', "Trinidad and Tobago")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Timor-Leste.*$)', "Timor-Leste")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Emirates.*$)', "United Arab Emirates (the)")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Papua.*$)', "Papua New Guinea")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bissau.*$)', "Guinea-Bissau")
df['Country Name'] = df['Country Name'].replace('Eq. Guinea','Equatorial Guinea')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Burma.*$)', "Myanmar")
df['Country Name'] = df['Country Name'].replace('C.A. Republic','Central African Republic (the)')
df['Country Name'] = df['Country Name'].replace('Ant.& Barb.','Antigua and Barbuda')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bosnia.*$)', "Bosnia and Herzegovina")
df['Country Name'] = df['Country Name'].replace('Domin. Rep.','Dominican Republic (the)')
df['Country Name'] = df['Country Name'].replace('Dominica (Commonwealth of)','Dominica')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*European Union.*$)', "European Union")
df['Country Name'] = df['Country Name'].replace('R. of Congo','Congo (the)')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Principe.*$)', "Sao Tome and Principe")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Solomon.*$)', "Solomon Islands")
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Vincent.*$)', "Saint Vincent and the Grenadines")
df['Country Name'] = df['Country Name'].replace('Curacao','Curaçao')
df['Country Name'] = df['Country Name'].replace('Reunion','Réunion')
df['Country Name'] = df['Country Name'].str.replace(r'(^.*Kosovo.*$)', "Kosovo (UNSCR 1244)")

df=df.merge(pop,how='outer',on='Country Name')
df

<ipython-input-142-709cf5e28ea8>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Bahamas.*$)', 'Bahamas (the)')
<ipython-input-142-709cf5e28ea8>:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r"(^.*Côte d'Ivoire.*$)", "Côte d'Ivoire")
<ipython-input-142-709cf5e28ea8>:35: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Czech.*$)', 'Czechia')
<ipython-input-142-709cf5e28ea8>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Country Name'] = df['Country Name'].str.replace(r'(^.*Dominican Re.*$)', 'Dominican Republic (the)')
<ipython-input-142-709cf5e28ea8>:37: FutureWarning: The default value of regex will change from Tru

,Country Name,Year,Indicator,data_col,higher_is_better,Sub-Pillar,1960,1961,1962,1963,...,2014,2015,2016,2017,2018,2019,2020,Country Code,Indicator Name,Indicator Code
0,Aruba,2020.0,"E-waste generated, kilograms per inhabitant",NaN,False,Digital Wellbeing,5.420800e+04,5.543400e+04,5.623400e+04,5.669900e+04,...,1.037760e+05,1.043390e+05,1.048650e+05,1.053610e+05,1.058460e+05,1.063100e+05,1.067660e+05,ABW,"Population, total",SP.POP.TOTL
1,Afghanistan,2020.0,"E-waste generated, kilograms per inhabitant",20000.0,False,Digital Wellbeing,8.996967e+06,9.169406e+06,9.351442e+06,9.543200e+06,...,3.337080e+07,3.441360e+07,3.538303e+07,3.629611e+07,3.717192e+07,3.804176e+07,3.892834e+07,AFG,"Population, total",SP.POP.TOTL
2,Angola,2020.0,"E-waste generated, kilograms per inhabitant",92000.0,False,Digital Wellbeing,5.454938e+06,5.531451e+06,5.608499e+06,5.679409e+06,...,2.694177e+07,2.788438e+07,2.884248e+07,2.981677e+07,3.080979e+07,3.182530e+07,3.286627e+07,AGO,"Population, total",SP.POP.TOTL
3,Albania,2020.0,"E-waste generated, kilograms per inhabitant",20000.0,False,Digital Wellbeing,1.608800e+06,1.659800e+06,1.711319e+06,1.762621e+06,...,2.889104e+06,2.880703e+06,2.876101e+06,2.873457e+06,2.866376e+06,2.854191e+06,2.837743e+06,ALB,"Population, total",SP.POP.TOTL
4,Andorra,2020.0,"E-waste generated, kilograms per inhabitant",NaN,False,Digital Wellbeing,1.341000e+04,1.437800e+04,1.537900e+04,1.640700e+04,...,7.921300e+04,7.799300e+04,7.729500e+04,7.699700e+04,7.700800e+04,7.714600e+04,7.726500e+04,AND,"Population, total",SP.POP.TOTL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Latin America & the Caribbean (IDA & IBRD coun...,NaN,NaN,NaN,NaN,NaN,2.097490e+08,2.156300e+08,2.216762e+08,2.278661e+08,...,5.999353e+08,6.062719e+08,6.125115e+08,6.186535e+08,6.246981e+08,6.306448e+08,6.364928e+08,TLA,"Population, total",SP.POP.TOTL
266,Middle East & North Africa (IDA & IBRD countries),NaN,NaN,NaN,NaN,NaN,9.755314e+07,1.001471e+08,1.028080e+08,1.055462e+08,...,3.534564e+08,3.597055e+08,3.659269e+08,3.721337e+08,3.783861e+08,3.847718e+08,3.913446e+08,TMN,"Population, total",SP.POP.TOTL
267,South Asia (IDA & IBRD),NaN,NaN,NaN,NaN,NaN,5.728395e+08,5.849397e+08,5.974944e+08,6.104983e+08,...,1.727632e+09,1.749417e+09,1.771187e+09,1.792883e+09,1.814455e+09,1.835777e+09,1.856882e+09,TSA,"Population, total",SP.POP.TOTL
268,Sub-Saharan Africa (IDA & IBRD countries),NaN,NaN,NaN,NaN,NaN,2.272332e+08,2.325670e+08,2.381216e+08,2.438934e+08,...,9.689584e+08,9.954585e+08,1.022530e+09,1.050163e+09,1.078320e+09,1.106958e+09,1.136047e+09,TSS,"Population, total",SP.POP.TOTL


In [143]:
df['new_data_col'] = (df['data_col']*1000)/(df['2020'])
df=df[df['Indicator'].notna()]
df=df[['Country Name','Year','Indicator','new_data_col','higher_is_better','Sub-Pillar']]

min_rank = df['new_data_col'].min()
max_rank = df['new_data_col'].max()

In [144]:
first = df['new_data_col'].quantile(1/5)
second = df['new_data_col'].quantile(2/5)
third = df['new_data_col'].quantile(3/5)
fourth = df['new_data_col'].quantile(4/5)
fifth = df['new_data_col'].quantile(5/5)
data = {'Min':min_rank,'Max':max_rank,'1':first,'2':second,'3':third,'4':fourth,'5':fifth}
quintile = pd.DataFrame(data, index=[0])
quintile

,Min,Max,1,2,3,4,5
0,0.003308,69.703486,0.831396,3.933627,6.825562,12.478495,69.703486


In [145]:
df.rename(columns={'new_data_col':'data_col'}, inplace=True)
# Convert the results into quintile
df['new_rank_score'] = pd.qcut(df['data_col'], 5, labels=[1,2,3,4,5])

# convert 1-5 rank into 1-6
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]

df['new_rank_score'] = df['new_rank_score'].apply(lambda row: convert_rank(row, old_min=1,old_max=5))
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)
df=df[df['new_rank_score'].notna()]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
1,Afghanistan,2020.0,"E-waste generated, kilograms per inhabitant",0.513765,5.9900,False,Digital Wellbeing
2,Angola,2020.0,"E-waste generated, kilograms per inhabitant",2.799223,4.7425,False,Digital Wellbeing
3,Albania,2020.0,"E-waste generated, kilograms per inhabitant",7.047855,2.2475,False,Digital Wellbeing
5,United Arab Emirates (the),2020.0,"E-waste generated, kilograms per inhabitant",13.548491,1.0000,False,Digital Wellbeing
6,Argentina,2020.0,"E-waste generated, kilograms per inhabitant",6.428400,3.4950,False,Digital Wellbeing
...,...,...,...,...,...,...,...
214,Kosovo (UNSCR 1244),2020.0,"E-waste generated, kilograms per inhabitant",0.012955,5.9900,False,Digital Wellbeing
215,Yemen,2020.0,"E-waste generated, kilograms per inhabitant",1.408169,4.7425,False,Digital Wellbeing
216,South Africa,2020.0,"E-waste generated, kilograms per inhabitant",5.412360,3.4950,False,Digital Wellbeing
217,Zambia,2020.0,"E-waste generated, kilograms per inhabitant",0.815929,5.9900,False,Digital Wellbeing


In [146]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
#df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)
df.insert(0,'Pillar','People')
df.to_csv('../non-index/people_{}_scores.csv'.format(indicator), index=False)

In [147]:
### 18. Automation-led unemployment

In [148]:
indicators[17]

# load data
indicator = indicators[17]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Automation-led unemployment
automation_led_unemployment


In [149]:
df.head(15)

,Country,No. of employees potentially automable (millions),Total employees (millions),Potential Rate of Automation (%)
0,Japan,35.60,63.90,55.71
1,Thailand,21.00,38.40,54.69
2,Senegal,2.20,4.07,54.00
3,Colombia,9.30,17.50,53.14
4,Peru,6.90,13.00,53.08
5,Taiwan,5.20,9.80,53.06
6,Kenya,7.40,14.20,52.11
7,South Korea,12.50,24.00,52.08
8,Sweden,2.10,4.04,52.00
9,Costa Rica,1.10,2.12,52.00


In [150]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital Wellbeing


In [151]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Potential Rate of Automation (%)'] 
df['Year'] = 2018
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)

In [152]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Japan,2018,Automation-led unemployment,55.71,3.210071,False,Digital Wellbeing
1,Thailand,2018,Automation-led unemployment,54.69,3.260969,False,Digital Wellbeing
2,Senegal,2018,Automation-led unemployment,54.00,3.295400,False,Digital Wellbeing
3,Colombia,2018,Automation-led unemployment,53.14,3.338314,False,Digital Wellbeing
4,Peru,2018,Automation-led unemployment,53.08,3.341308,False,Digital Wellbeing
5,Taiwan,2018,Automation-led unemployment,53.06,3.342306,False,Digital Wellbeing
6,Kenya,2018,Automation-led unemployment,52.11,3.389711,False,Digital Wellbeing
7,South Korea,2018,Automation-led unemployment,52.08,3.391208,False,Digital Wellbeing
8,Sweden,2018,Automation-led unemployment,52.00,3.395200,False,Digital Wellbeing
9,Costa Rica,2018,Automation-led unemployment,52.00,3.395200,False,Digital Wellbeing


In [153]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [154]:
### 19. Cyberbullying

In [155]:
indicators[18]

# load data
indicator = indicators[18]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Cyberbullying
cyberbullying_rate


In [156]:
df.head(15)

# Need to move the top row down further

,2011,2016,2018,Country
0,32,32,37,India
1,20,19,29,Brazil
2,15,34,26,United States
3,12,13,25,Belgium
4,10,25,26,South Africa
5,--,--,23,Malaysia
6,14,20,23,Sweden
7,18,17,20,Canada
8,5,14,20,Turkey
9,18,17,19,Saudi Arabia


In [157]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital Wellbeing


In [158]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['2018'] 
df['Year'] = 2018
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# invert since to put as higher is not better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)

In [159]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,India,2018,Cyberbullying,37,4.1437,False,Digital Wellbeing
1,Brazil,2018,Cyberbullying,29,4.5429,False,Digital Wellbeing
2,United States,2018,Cyberbullying,26,4.6926,False,Digital Wellbeing
3,Belgium,2018,Cyberbullying,25,4.7425,False,Digital Wellbeing
4,South Africa,2018,Cyberbullying,26,4.6926,False,Digital Wellbeing
5,Malaysia,2018,Cyberbullying,23,4.8423,False,Digital Wellbeing
6,Sweden,2018,Cyberbullying,23,4.8423,False,Digital Wellbeing
7,Canada,2018,Cyberbullying,20,4.9920,False,Digital Wellbeing
8,Turkey,2018,Cyberbullying,20,4.9920,False,Digital Wellbeing
9,Saudi Arabia,2018,Cyberbullying,19,5.0419,False,Digital Wellbeing


In [160]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [161]:
### 20. Global Wellbeing Initiative

In [162]:
indicators[19]

# load data
indicator = indicators[19]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# Strange the global_wellbeing_initiative is clearly in the data manifest and the link works well 
# but it has not been moved to the processed folder

Global Wellbeing Initiative (World Happiness Index)
global_wellbeing_initiative


In [163]:
df.head(15)

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,...,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual,data_country,data_year
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.000,0.949,...,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253,NaN,NaN
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.700,0.946,...,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868,NaN,NaN
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.400,0.919,...,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839,NaN,NaN
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.000,0.955,...,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967,NaN,NaN
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.400,0.913,...,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798,NaN,NaN
5,Norway,Western Europe,7.392,0.035,7.462,7.323,11.053,0.954,73.300,0.960,...,2.43,1.543,1.108,0.782,0.703,0.249,0.427,2.580,NaN,NaN
6,Sweden,Western Europe,7.363,0.036,7.433,7.293,10.867,0.934,72.700,0.945,...,2.43,1.478,1.062,0.763,0.685,0.244,0.448,2.683,NaN,NaN
7,Luxembourg,Western Europe,7.324,0.037,7.396,7.252,11.647,0.908,72.600,0.907,...,2.43,1.751,1.003,0.760,0.639,0.166,0.353,2.653,NaN,NaN
8,New Zealand,North America and ANZ,7.277,0.040,7.355,7.198,10.643,0.948,73.400,0.929,...,2.43,1.400,1.094,0.785,0.665,0.276,0.445,2.612,NaN,NaN
9,Austria,Western Europe,7.268,0.036,7.337,7.198,10.906,0.934,73.300,0.908,...,2.43,1.492,1.062,0.782,0.640,0.215,0.292,2.784,NaN,NaN


In [164]:
subpillars[3]
subpillar = subpillars[3]
print(subpillar)

Digital Wellbeing


In [165]:
# create standard columns
df.rename(columns={'Country name':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Ladder score'] 
df['Year'] = 2021
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()


df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_c(row, old_min=0,old_max=10))


In [166]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Finland,2021,Global Wellbeing Initiative (World Happiness I...,7.842,4.913158,True,Digital Wellbeing
1,Denmark,2021,Global Wellbeing Initiative (World Happiness I...,7.620,4.802380,True,Digital Wellbeing
2,Switzerland,2021,Global Wellbeing Initiative (World Happiness I...,7.571,4.777929,True,Digital Wellbeing
3,Iceland,2021,Global Wellbeing Initiative (World Happiness I...,7.554,4.769446,True,Digital Wellbeing
4,Netherlands,2021,Global Wellbeing Initiative (World Happiness I...,7.464,4.724536,True,Digital Wellbeing
...,...,...,...,...,...,...,...
144,Lesotho,2021,Global Wellbeing Initiative (World Happiness I...,3.512,2.752488,True,Digital Wellbeing
145,Botswana,2021,Global Wellbeing Initiative (World Happiness I...,3.467,2.730033,True,Digital Wellbeing
146,Rwanda,2021,Global Wellbeing Initiative (World Happiness I...,3.415,2.704085,True,Digital Wellbeing
147,Zimbabwe,2021,Global Wellbeing Initiative (World Happiness I...,3.145,2.569355,True,Digital Wellbeing


In [167]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [168]:
### 21. Financial Inclusiveness

In [169]:
indicators[20]

# load data
indicator = indicators[20]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)


df = pd.read_csv('../../processed/{}.csv'.format(bf))
# Strange the global_wellbeing_initiative is clearly in the data manifest and the link works well 
# but it has not been moved to the processed folder

Financial Inclusiveness
financial_inclusiveness


In [170]:
df.head(15)

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,28%,34%,36%,15%,23%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,38%,43%,44%,29%,34%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,40%,42%,53%,27%,38%,...,2%,6%,1%,1%,4%,0%,4%,2%,NaN,NaN
8,2011,ARB,Arab world,NaN,NaN,22%,30%,33%,11%,14%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,ARB,Arab world,NaN,NaN,30%,38%,42%,18%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [172]:
df = df[(df.Year == 2017)]

# Must convert the data to float by removing the % sign
df['Account (% age 15+)'] = df['Account (% age 15+)'].str.replace('%','')
df['Account (% age 15+)'] = df['Account (% age 15+)'].astype(float)

In [173]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Account (% age 15+)'] 
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [174]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Afghanistan,2017,Financial Inclusiveness,15.0,1.7485,True,Usage and ownership
7,Albania,2017,Financial Inclusiveness,40.0,2.9960,True,Usage and ownership
10,Arab world,2017,Financial Inclusiveness,37.0,2.8463,True,Usage and ownership
13,United Arab Emirates,2017,Financial Inclusiveness,88.0,5.3912,True,Usage and ownership
16,Argentina,2017,Financial Inclusiveness,49.0,3.4451,True,Usage and ownership
...,...,...,...,...,...,...,...
479,World,2017,Financial Inclusiveness,69.0,4.4431,True,Usage and ownership
482,Kosovo,2017,Financial Inclusiveness,52.0,3.5948,True,Usage and ownership
487,South Africa,2017,Financial Inclusiveness,69.0,4.4431,True,Usage and ownership
490,Zambia,2017,Financial Inclusiveness,46.0,3.2954,True,Usage and ownership


In [175]:
# output scores
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [176]:
### 22. E-commerce activity (% of individuals buying online and frequency)

In [177]:
indicators[21]

# load data
indicator = indicators[21]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

# Added _ between spaces to see if this changes anything

E-commerce activity (% of individuals buying online and frequency)
individuals_buying_online_frequency


In [178]:
df.head(15)

# Move the top row down further

,RANK,COUNTRY/ECONOMY,VALUE (%),SCORE,Year
0,1.0,Denmark,77.97,100.00,2017
1,2.0,Netherlands,75.72,97.11,2017
2,3.0,Norway,75.63,96.99,2017
3,4.0,United Kingdom,74.74,95.86,2017
4,5.0,"Korea, Rep.",72.47,92.94,2017
5,6.0,Sweden,71.65,91.90,2017
6,7.0,United States,70.43,90.33,2017
7,8.0,New Zealand,69.11,88.63,2017
8,9.0,Canada,68.57,87.95,2017
9,10.0,Australia,67.69,86.81,2017


In [179]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [180]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['SCORE'] 
df['Country Name'] = df['COUNTRY/ECONOMY']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

In [181]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,Denmark,2017,E-commerce activity (% of individuals buying o...,100.00,5.990000,True,Usage and ownership
1,Netherlands,2017,E-commerce activity (% of individuals buying o...,97.11,5.845789,True,Usage and ownership
2,Norway,2017,E-commerce activity (% of individuals buying o...,96.99,5.839801,True,Usage and ownership
3,United Kingdom,2017,E-commerce activity (% of individuals buying o...,95.86,5.783414,True,Usage and ownership
4,"Korea, Rep.",2017,E-commerce activity (% of individuals buying o...,92.94,5.637706,True,Usage and ownership
...,...,...,...,...,...,...,...
129,Iceland,2017,E-commerce activity (% of individuals buying o...,NaN,NaN,True,Usage and ownership
130,Jamaica,2017,E-commerce activity (% of individuals buying o...,NaN,NaN,True,Usage and ownership
131,Oman,2017,E-commerce activity (% of individuals buying o...,NaN,NaN,True,Usage and ownership
132,Qatar,2017,E-commerce activity (% of individuals buying o...,NaN,NaN,True,Usage and ownership


In [182]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [183]:
### 23. Mobile Device Penetration

In [184]:
indicators[22]

# load data
indicator = indicators[22]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))


Mobile Device Penetration
ITU_database


In [185]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [186]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [187]:
df = df[(df['Indicator name'] == 'Individuals owning a mobile phone (%)')]
df = df[(df.Year == 2019)]
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
5753,Angola,Africa,AGO,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5754,Benin,Africa,BEN,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5755,Botswana,Africa,BWA,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5756,Burkina Faso,Africa,BFA,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5757,Burundi,Africa,BDI,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5943,Suriname,The Americas,SUR,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5944,Trinidad and Tobago,The Americas,TTO,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5945,United States,The Americas,USA,Individuals owning a mobile phone (%),2019.0,NaN,NaN,NaN
5946,Uruguay,The Americas,URY,Individuals owning a mobile phone (%),2019.0,83.282233,NaN,NaN


In [188]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [189]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5753,Angola,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and ownership
5754,Benin,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and ownership
5755,Botswana,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and ownership
5756,Burkina Faso,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and ownership
5757,Burundi,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and ownership
...,...,...,...,...,...,...,...
5943,Suriname,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and ownership
5944,Trinidad and Tobago,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and ownership
5945,United States,2019.0,Mobile Device Penetration,NaN,NaN,True,Usage and ownership
5946,Uruguay,2019.0,Mobile Device Penetration,83.282233,5.155783,True,Usage and ownership


In [190]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [191]:
### 24. Mobile Device Penetration (female)

In [192]:
indicators[23]

# load data
indicator = indicators[23]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Mobile Device Penetration (female)
ITU_database


In [193]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [194]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [195]:
df = df[(df.Year==2019)]
df = df[(df['Indicator name'] == 'Female mobile phone ownership as a % of total female population')]

In [196]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [197]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
1763,Angola,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and ownership
1764,Benin,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and ownership
1765,Botswana,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and ownership
1766,Burkina Faso,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and ownership
1767,Burundi,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and ownership
...,...,...,...,...,...,...,...
1953,Suriname,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and ownership
1954,Trinidad and Tobago,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and ownership
1955,United States,2019.0,Mobile Device Penetration (female),NaN,NaN,True,Usage and ownership
1956,Uruguay,2019.0,Mobile Device Penetration (female),84.173624,5.200264,True,Usage and ownership


In [198]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [199]:
### 25. Internet Usage

In [200]:
indicators[24]

# load data
indicator = indicators[24]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Internet Usage
ITU_database


In [201]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [202]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [203]:
df = df[(df['Indicator name'] == 'Individuals using the Internet, total (%)')]
df = df[(df.Year == 2019)]

In [204]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# convert 1-190 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

In [205]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
47607,Cabo Verde,2019.0,Internet Usage,61.943398,4.090976,True,Usage and ownership
47608,Central African Rep.,2019.0,Internet Usage,NaN,NaN,True,Usage and ownership
47609,Congo (Rep. of the),2019.0,Internet Usage,NaN,NaN,True,Usage and ownership
47610,Côte d'Ivoire,2019.0,Internet Usage,36.288955,2.810819,True,Usage and ownership
47611,Equatorial Guinea,2019.0,Internet Usage,NaN,NaN,True,Usage and ownership
...,...,...,...,...,...,...,...
47739,Saint Vincent and the Grenadines,2019.0,Internet Usage,NaN,NaN,True,Usage and ownership
47740,Trinidad and Tobago,2019.0,Internet Usage,NaN,NaN,True,Usage and ownership
47741,United States,2019.0,Internet Usage,89.430285,5.462571,True,Usage and ownership
47742,Uruguay,2019.0,Internet Usage,83.351534,5.159242,True,Usage and ownership


In [206]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [207]:
### 26. Households with a computer and with Internet Access

In [208]:
indicators[25]

# load data
indicator = indicators[25]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Households with a computer and with Internet access
ITU_database


In [209]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [210]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [211]:
df = df[(df['Indicator name'] == 'Households with Internet access at home (%)')]
df = df[(df.Year == 2020)]
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
45188,Mauritius,Africa,MUS,Households with Internet access at home (%),2020.0,72.642477,NaN,NaN
45189,Bahrain,Arab States,BHR,Households with Internet access at home (%),2020.0,99.671115,NaN,NaN
45190,Egypt,Arab States,EGY,Households with Internet access at home (%),2020.0,73.011528,NaN,NaN
45191,Kuwait,Arab States,KWT,Households with Internet access at home (%),2020.0,99.399945,NaN,NaN
45192,Morocco,Arab States,MAR,Households with Internet access at home (%),2020.0,84.520014,NaN,NaN
...,...,...,...,...,...,...,...,...
45250,Cuba,The Americas,CUB,Households with Internet access at home (%),2020.0,33.309090,NaN,NaN
45251,Ecuador,The Americas,ECU,Households with Internet access at home (%),2020.0,53.209495,NaN,NaN
45252,Mexico,The Americas,MEX,Households with Internet access at home (%),2020.0,60.553970,NaN,NaN
45253,Paraguay,The Americas,PRY,Households with Internet access at home (%),2020.0,36.533608,NaN,NaN


In [212]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [213]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
45188,Mauritius,2020.0,Households with a computer and with Internet a...,72.642477,4.624860,True,Usage and ownership
45189,Bahrain,2020.0,Households with a computer and with Internet a...,99.671115,5.973589,True,Usage and ownership
45190,Egypt,2020.0,Households with a computer and with Internet a...,73.011528,4.643275,True,Usage and ownership
45191,Kuwait,2020.0,Households with a computer and with Internet a...,99.399945,5.960057,True,Usage and ownership
45192,Morocco,2020.0,Households with a computer and with Internet a...,84.520014,5.217549,True,Usage and ownership
...,...,...,...,...,...,...,...
45250,Cuba,2020.0,Households with a computer and with Internet a...,33.309090,2.662124,True,Usage and ownership
45251,Ecuador,2020.0,Households with a computer and with Internet a...,53.209495,3.655154,True,Usage and ownership
45252,Mexico,2020.0,Households with a computer and with Internet a...,60.553970,4.021643,True,Usage and ownership
45253,Paraguay,2020.0,Households with a computer and with Internet a...,36.533608,2.823027,True,Usage and ownership


In [214]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [215]:
### 27. % Social media penetration

In [216]:
indicators[26]

# load data
indicator = indicators[26]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Social media penetration
social_media_penetration


In [217]:
df.head(15)

,Country Name,Active social network penetration in selected countries and territories as of January 2021,Unnamed: 2
0,UAE,99.0,NaN
1,South Korea,89.3,NaN
2,Taiwan,88.1,NaN
3,Netherlands,88.0,NaN
4,Malaysia,86.0,NaN
5,Hong Kong,85.6,NaN
6,Canada,84.9,NaN
7,Singapore,84.4,NaN
8,Denmark,83.6,NaN
9,Sweden,82.1,NaN


In [218]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [219]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Active social network penetration in selected countries and territories as of January 2021'] 
df['Year'] = 2021
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [220]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,UAE,2021,Social media penetration,99.0,5.94010,True,Usage and ownership
1,South Korea,2021,Social media penetration,89.3,5.45607,True,Usage and ownership
2,Taiwan,2021,Social media penetration,88.1,5.39619,True,Usage and ownership
3,Netherlands,2021,Social media penetration,88.0,5.39120,True,Usage and ownership
4,Malaysia,2021,Social media penetration,86.0,5.29140,True,Usage and ownership
5,Hong Kong,2021,Social media penetration,85.6,5.27144,True,Usage and ownership
6,Canada,2021,Social media penetration,84.9,5.23651,True,Usage and ownership
7,Singapore,2021,Social media penetration,84.4,5.21156,True,Usage and ownership
8,Denmark,2021,Social media penetration,83.6,5.17164,True,Usage and ownership
9,Sweden,2021,Social media penetration,82.1,5.09679,True,Usage and ownership


In [221]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [222]:
### 28. % of population using Facebook

In [223]:
indicators[27]

# load data
indicator = indicators[27]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using Facebook
FB_users


In [224]:
df.head(15)

,Country,Facebook Users,Population in Thousands (2021),Percentage of Facebook Users
0,India,"251,000,000","1,393,409",18.01
1,United States,"240,000,000","332,915",72.09
2,Brazil,"139,000,000","213,993",64.96
3,Indonesia,"136,960,000","276,362",49.56
4,Mexico,"78,000,000","130,262",59.88
5,Philippines,"71,760,000","111,047",64.62
6,Vietnam,"66,720,000","98,169",67.96
7,Thailand,"46,000,000","69,951",65.76
8,United Kingdom,"44,000,000","68,207",64.51
9,Turkey,"44,000,000","85,043",51.74


In [225]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [226]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Percentage of Facebook Users'] 
df['Year'] = 2021
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [227]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,India,2021,% of population using Facebook,18.01,1.898699,True,Usage and ownership
1,United States,2021,% of population using Facebook,72.09,4.597291,True,Usage and ownership
2,Brazil,2021,% of population using Facebook,64.96,4.241504,True,Usage and ownership
3,Indonesia,2021,% of population using Facebook,49.56,3.473044,True,Usage and ownership
4,Mexico,2021,% of population using Facebook,59.88,3.988012,True,Usage and ownership
...,...,...,...,...,...,...,...
223,Tuvalu,2021,% of population using Facebook,14.17,1.707083,True,Usage and ownership
224,Niue,2021,% of population using Facebook,41.00,3.045900,True,Usage and ownership
225,Tokelau,2021,% of population using Facebook,41.00,3.045900,True,Usage and ownership
226,Vatican City,2021,% of population using Facebook,2.00,1.099800,True,Usage and ownership


In [228]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [229]:
### 29. Gender gap for social media use

In [230]:
indicators[28]

# load data
indicator = indicators[28]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Gender gap for social media use
gender_gaps


In [231]:
df.head(15)

,ISO Code,Country,Region,Year,Gender parity in account ownership,Gender gap in social media use,Gender gap in mobile ownership
0,AFG,Afghanistan,South Asia,2014,11.109999,0.000000,0.000000
1,AFG,Afghanistan,South Asia,2015,14.087173,0.000000,0.000000
2,AFG,Afghanistan,South Asia,2016,17.064350,0.000000,0.000000
3,AFG,Afghanistan,South Asia,2017,20.041523,0.000000,0.000000
4,AFG,Afghanistan,South Asia,2018,20.041523,0.000000,0.000000
5,AFG,Afghanistan,South Asia,2019,20.041523,0.000000,0.000000
6,AGO,Angola,Sub-Saharan Africa,2014,55.237572,46.428570,51.645042
7,AGO,Angola,Sub-Saharan Africa,2015,55.237572,46.428570,57.001461
8,AGO,Angola,Sub-Saharan Africa,2016,55.237572,46.428570,63.725491
9,AGO,Angola,Sub-Saharan Africa,2017,55.237572,38.646553,53.966476


In [232]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [233]:
df = df[df.Year==2019]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Gender gap in social media use'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [234]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Gender gap for social media use,0.000000,1.000000,True,Usage and ownership
11,Angola,2019,Gender gap for social media use,49.552494,3.472669,True,Usage and ownership
17,Albania,2019,Gender gap for social media use,41.406120,3.066165,True,Usage and ownership
23,United Arab Emirates,2019,Gender gap for social media use,13.890497,1.693136,True,Usage and ownership
29,Argentina,2019,Gender gap for social media use,100.000000,5.990000,True,Usage and ownership
...,...,...,...,...,...,...,...
995,Samoa,2019,Gender gap for social media use,100.000000,5.990000,True,Usage and ownership
1001,Yemen,2019,Gender gap for social media use,0.000000,1.000000,True,Usage and ownership
1007,South Africa,2019,Gender gap for social media use,96.334473,5.807090,True,Usage and ownership
1013,Zambia,2019,Gender gap for social media use,59.793156,3.983678,True,Usage and ownership


In [235]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [236]:
### 30. % of population using digital financial services

In [237]:
indicators[29]

# load data
indicator = indicators[29]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population using digital financial services
population_digital_financial_services


In [238]:
df.head(15)

# May have to convert the data to csv file as the original file is missing many important columns

,Year,ISO,Country Name,Region,Income Group,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,2011,AFG,Afghanistan,South Asia,Low income,9%,15%,15%,2%,3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,AFG,Afghanistan,South Asia,Low income,10%,16%,15%,4%,4%,...,0%,0%,0%,0%,0%,0%,1%,0%,NaN,NaN
2,2017,AFG,Afghanistan,South Asia,Low income,15%,23%,25%,4%,7%,...,1%,0%,1%,0%,2%,0%,1%,1%,NaN,NaN
3,2011,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,39%,39%,46%,31%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,AGO,Angola,Sub-Saharan Africa (excluding high income),Lower middle income,29%,36%,36%,12%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,28%,34%,36%,15%,23%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2014,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,38%,43%,44%,29%,34%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2017,ALB,Albania,Europe & Central Asia (excluding high income),Upper middle income,40%,42%,53%,27%,38%,...,2%,6%,1%,1%,4%,0%,4%,2%,NaN,NaN
8,2011,ARB,Arab world,NaN,NaN,22%,30%,33%,11%,14%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014,ARB,Arab world,NaN,NaN,30%,38%,42%,18%,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [239]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [240]:
df = df[(df.Year == 2017)]

# Must convert the data to float by removing the % sign
df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'] = df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'].str.replace('%','')
df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'] = df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'].astype(float)

In [241]:
# create standard columns
df.rename(columns={'Country':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Used the internet to pay bills or to buy something online in the past year (% age 15+)'] 
df['Sub-Pillar'] = subpillar


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-20 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [242]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df 

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
2,Afghanistan,2017,% of population using digital financial services,1.0,1.0499,True,Usage and ownership
7,Albania,2017,% of population using digital financial services,7.0,1.3493,True,Usage and ownership
10,Arab world,2017,% of population using digital financial services,9.0,1.4491,True,Usage and ownership
13,United Arab Emirates,2017,% of population using digital financial services,60.0,3.9940,True,Usage and ownership
16,Argentina,2017,% of population using digital financial services,19.0,1.9481,True,Usage and ownership
...,...,...,...,...,...,...,...
479,World,2017,% of population using digital financial services,29.0,2.4471,True,Usage and ownership
482,Kosovo,2017,% of population using digital financial services,15.0,1.7485,True,Usage and ownership
487,South Africa,2017,% of population using digital financial services,14.0,1.6986,True,Usage and ownership
490,Zambia,2017,% of population using digital financial services,11.0,1.5489,True,Usage and ownership


In [243]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [244]:
### 31. Mobile Broadband Pricing (pre-paid)

In [245]:
indicators[30]

# load data
indicator = indicators[30]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Mobile Broadband Pricing (pre-paid)
ITU_database


In [246]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [247]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [248]:
df = df[df.Year==2019]
df = df[df['Indicator name'] == 'Mobile broadband basket as a % of GNI p.c.']
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
17974,Angola,Africa,AGO,Mobile broadband basket as a % of GNI p.c.,2019.0,5.63,NaN,NaN
17975,Benin,Africa,BEN,Mobile broadband basket as a % of GNI p.c.,2019.0,6.21,NaN,NaN
17976,Botswana,Africa,BWA,Mobile broadband basket as a % of GNI p.c.,2019.0,2.87,NaN,NaN
17977,Burkina Faso,Africa,BFA,Mobile broadband basket as a % of GNI p.c.,2019.0,19.63,NaN,NaN
17978,Burundi,Africa,BDI,Mobile broadband basket as a % of GNI p.c.,2019.0,20.43,NaN,NaN
...,...,...,...,...,...,...,...,...
18161,Saint Vincent and the Grenadines,The Americas,VCT,Mobile broadband basket as a % of GNI p.c.,2019.0,4.59,NaN,NaN
18162,Suriname,The Americas,SUR,Mobile broadband basket as a % of GNI p.c.,2019.0,1.93,NaN,NaN
18163,Trinidad and Tobago,The Americas,TTO,Mobile broadband basket as a % of GNI p.c.,2019.0,3.07,NaN,NaN
18164,United States,The Americas,USA,Mobile broadband basket as a % of GNI p.c.,2019.0,0.42,NaN,NaN


In [249]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)

In [250]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
17974,Angola,2019.0,Mobile Broadband Pricing (pre-paid),5.63,5.709063,True,Usage and ownership
17975,Benin,2019.0,Mobile Broadband Pricing (pre-paid),6.21,5.680121,True,Usage and ownership
17976,Botswana,2019.0,Mobile Broadband Pricing (pre-paid),2.87,5.846787,True,Usage and ownership
17977,Burkina Faso,2019.0,Mobile Broadband Pricing (pre-paid),19.63,5.010463,True,Usage and ownership
17978,Burundi,2019.0,Mobile Broadband Pricing (pre-paid),20.43,4.970543,True,Usage and ownership
...,...,...,...,...,...,...,...
18161,Saint Vincent and the Grenadines,2019.0,Mobile Broadband Pricing (pre-paid),4.59,5.760959,True,Usage and ownership
18162,Suriname,2019.0,Mobile Broadband Pricing (pre-paid),1.93,5.893693,True,Usage and ownership
18163,Trinidad and Tobago,2019.0,Mobile Broadband Pricing (pre-paid),3.07,5.836807,True,Usage and ownership
18164,United States,2019.0,Mobile Broadband Pricing (pre-paid),0.42,5.969042,True,Usage and ownership


In [251]:
### 32. Tax as % of total cost of mobile ownership

In [252]:
indicators[31]

# load data
indicator = indicators[31]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Tax as % of total cost of mobile ownership
tax_percent_mobile_ownership


In [253]:
df.head(15)

,ISO Code,Country,Region,Year,Cluster,Index,Infrastructure,Affordability,Consumer Readiness,Content and Services,...,Gender gap in mobile ownership,TLDs per capita,E-Government Score,Mobile Social Media Penetration,Apps developed per person,Number of apps in national language,Accessibility of top ranked apps,Cybersecurity Index,data_country,data_year
0,AFG,Afghanistan,South Asia,2014,Discoverer,22.12,21.74,31.79,24.40,14.19,...,0.00,39.55,18.11,3.28,20.98,2.44,4.37,26.50,NaN,NaN
1,AFG,Afghanistan,South Asia,2015,Discoverer,22.99,22.82,30.81,25.28,15.71,...,0.00,39.57,24.27,4.36,22.93,2.79,8.03,25.83,NaN,NaN
2,AFG,Afghanistan,South Asia,2016,Discoverer,23.71,26.92,26.75,26.07,16.83,...,0.00,39.58,30.43,6.73,30.31,2.85,5.90,25.17,NaN,NaN
3,AFG,Afghanistan,South Asia,2017,Discoverer,25.82,33.54,27.22,28.56,17.04,...,0.00,39.47,30.50,7.78,31.62,2.91,6.15,24.50,NaN,NaN
4,AFG,Afghanistan,South Asia,2018,Discoverer,28.39,30.91,42.64,29.24,16.87,...,0.00,39.39,30.56,8.54,36.54,2.96,8.66,17.70,NaN,NaN
5,AFG,Afghanistan,South Asia,2019,Discoverer,28.94,32.34,41.53,29.72,17.58,...,0.00,39.41,41.18,9.39,39.36,3.00,5.72,17.70,NaN,NaN
6,AGO,Angola,Sub-Saharan Africa,2014,Discoverer,32.78,25.99,35.99,44.33,27.85,...,51.65,0.00,29.92,3.99,22.74,53.33,49.09,8.80,NaN,NaN
7,AGO,Angola,Sub-Saharan Africa,2015,Emerging,37.18,33.09,42.01,45.41,30.27,...,57.00,0.31,32.35,5.26,22.12,55.08,58.33,8.47,NaN,NaN
8,AGO,Angola,Sub-Saharan Africa,2016,Emerging,39.85,37.80,44.74,46.47,32.11,...,63.73,0.00,34.78,6.27,27.46,56.52,61.73,8.13,NaN,NaN
9,AGO,Angola,Sub-Saharan Africa,2017,Emerging,42.89,48.60,47.94,46.32,31.36,...,53.97,0.00,37.88,4.26,31.94,57.06,55.09,7.80,NaN,NaN


In [254]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [255]:
df = df[df.Year==2019]

# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = False
df['Indicator'] = indicator
df['data_col'] = df['Tax as a % of TCMO'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

# invert since to put as higher is  better
df['new_rank_score'] = df['new_rank_score'].apply(lambda row: (5.99-row)+1)

In [256]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Tax as % of total cost of mobile ownership,79.16,2.039916,False,Usage and ownership
11,Angola,2019,Tax as % of total cost of mobile ownership,65.83,2.705083,False,Usage and ownership
17,Albania,2019,Tax as % of total cost of mobile ownership,50.00,3.495000,False,Usage and ownership
23,United Arab Emirates,2019,Tax as % of total cost of mobile ownership,82.31,1.882731,False,Usage and ownership
29,Argentina,2019,Tax as % of total cost of mobile ownership,5.23,5.729023,False,Usage and ownership
...,...,...,...,...,...,...,...
995,Samoa,2019,Tax as % of total cost of mobile ownership,62.50,2.871250,False,Usage and ownership
1001,Yemen,2019,Tax as % of total cost of mobile ownership,75.00,2.247500,False,Usage and ownership
1007,South Africa,2019,Tax as % of total cost of mobile ownership,62.50,2.871250,False,Usage and ownership
1013,Zambia,2019,Tax as % of total cost of mobile ownership,17.88,5.097788,False,Usage and ownership


In [257]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [258]:
### 33. % of population with a smartphone

In [259]:
indicators[32]

# load data
indicator = indicators[32]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population with a smartphone
population_with_smartphones


In [260]:
df.head(15)

,Unnamed: 0,Country,Total Population,Smartphone Penetration Rate,Smartphone Users
0,1,China,1.44B,0.64,918.45M
1,2,India,1.38B,0.32,439.42M
2,3,United States,331M,0.82,270M
3,4,Indonesia,273.52M,0.59,160.23M
4,5,Brazil,212.56M,0.51,109.34M
5,6,Russia,145.93M,0.69,99.93M
6,7,Japan,126.48M,0.63,80M
7,8,Mexico,128.93M,0.54,70.14M
8,9,Germany,83.78M,0.78,65.24M
9,10,Vietnam,97.34M,0.63,61.37M


In [261]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [262]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Smartphone Penetration Rate'] 
df['Country Name'] = df['Country']
df['Year'] = 2020
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank_b(row,old_min=0,old_max=1))

In [263]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
0,China,2020,% of population with a smartphone,0.64,4.1936,True,Usage and ownership
1,India,2020,% of population with a smartphone,0.32,2.5968,True,Usage and ownership
2,United States,2020,% of population with a smartphone,0.82,5.0918,True,Usage and ownership
3,Indonesia,2020,% of population with a smartphone,0.59,3.9441,True,Usage and ownership
4,Brazil,2020,% of population with a smartphone,0.51,3.5449,True,Usage and ownership
5,Russia,2020,% of population with a smartphone,0.69,4.4431,True,Usage and ownership
6,Japan,2020,% of population with a smartphone,0.63,4.1437,True,Usage and ownership
7,Mexico,2020,% of population with a smartphone,0.54,3.6946,True,Usage and ownership
8,Germany,2020,% of population with a smartphone,0.78,4.8922,True,Usage and ownership
9,Vietnam,2020,% of population with a smartphone,0.63,4.1437,True,Usage and ownership


In [264]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [265]:
### 34. Gender gap in internet usage

In [266]:
indicators[33]

# load data
indicator = indicators[33]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Gender gap in internet usage
ITU_database


In [267]:
df.head(15)

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
0,Angola,Africa,AGO,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
1,Benin,Africa,BEN,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
2,Botswana,Africa,BWA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
3,Burkina Faso,Africa,BFA,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
4,Burundi,Africa,BDI,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
5,Cabo Verde,Africa,CPV,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
6,Cameroon,Africa,CMR,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
7,Central African Rep.,Africa,CAF,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
8,Chad,Africa,TCD,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN
9,Congo (Rep. of the),Africa,COG,Female mobile phone ownership as a % of total ...,2010.0,NaN,NaN,NaN


In [268]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [269]:
df = df[(df['Indicator name'] == 'Female Internet users as a % of total female population')]
df = df[(df.Year == 2019)]
df

,Country,Region,ISO,Indicator name,Year,Value,data_country,data_year
40307,Cabo Verde,Africa,CPV,Female Internet users as a % of total female p...,2019.0,60.965017,NaN,NaN
40308,Côte d'Ivoire,Africa,CIV,Female Internet users as a % of total female p...,2019.0,32.924445,NaN,NaN
40309,Kenya,Africa,KEN,Female Internet users as a % of total female p...,2019.0,20.125122,NaN,NaN
40310,Lesotho,Africa,LSO,Female Internet users as a % of total female p...,2019.0,44.922768,NaN,NaN
40311,Mauritius,Africa,MUS,Female Internet users as a % of total female p...,2019.0,60.130645,NaN,NaN
...,...,...,...,...,...,...,...,...
40388,Mexico,The Americas,MEX,Female Internet users as a % of total female p...,2019.0,68.574653,NaN,NaN
40389,Panama,The Americas,PAN,Female Internet users as a % of total female p...,2019.0,63.811494,NaN,NaN
40390,Paraguay,The Americas,PRY,Female Internet users as a % of total female p...,2019.0,69.017230,NaN,NaN
40391,Peru,The Americas,PER,Female Internet users as a % of total female p...,2019.0,56.992144,NaN,NaN


In [270]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Value'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [271]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
40307,Cabo Verde,2019.0,Gender gap in internet usage,60.965017,4.042154,True,Usage and ownership
40308,Côte d'Ivoire,2019.0,Gender gap in internet usage,32.924445,2.642930,True,Usage and ownership
40309,Kenya,2019.0,Gender gap in internet usage,20.125122,2.004244,True,Usage and ownership
40310,Lesotho,2019.0,Gender gap in internet usage,44.922768,3.241646,True,Usage and ownership
40311,Mauritius,2019.0,Gender gap in internet usage,60.130645,4.000519,True,Usage and ownership
...,...,...,...,...,...,...,...
40388,Mexico,2019.0,Gender gap in internet usage,68.574653,4.421875,True,Usage and ownership
40389,Panama,2019.0,Gender gap in internet usage,63.811494,4.184194,True,Usage and ownership
40390,Paraguay,2019.0,Gender gap in internet usage,69.017230,4.443960,True,Usage and ownership
40391,Peru,2019.0,Gender gap in internet usage,56.992144,3.843908,True,Usage and ownership


In [272]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [273]:
### 35. Gender gap in mobile usage

In [274]:
indicators[34]

# load data
indicator = indicators[34]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Gender gap in mobile usage
gender_gaps


In [275]:
df.head(15)

,ISO Code,Country,Region,Year,Gender parity in account ownership,Gender gap in social media use,Gender gap in mobile ownership
0,AFG,Afghanistan,South Asia,2014,11.109999,0.000000,0.000000
1,AFG,Afghanistan,South Asia,2015,14.087173,0.000000,0.000000
2,AFG,Afghanistan,South Asia,2016,17.064350,0.000000,0.000000
3,AFG,Afghanistan,South Asia,2017,20.041523,0.000000,0.000000
4,AFG,Afghanistan,South Asia,2018,20.041523,0.000000,0.000000
5,AFG,Afghanistan,South Asia,2019,20.041523,0.000000,0.000000
6,AGO,Angola,Sub-Saharan Africa,2014,55.237572,46.428570,51.645042
7,AGO,Angola,Sub-Saharan Africa,2015,55.237572,46.428570,57.001461
8,AGO,Angola,Sub-Saharan Africa,2016,55.237572,46.428570,63.725491
9,AGO,Angola,Sub-Saharan Africa,2017,55.237572,38.646553,53.966476


In [276]:
subpillars[1]
subpillar = subpillars[1]
print(subpillar)

Usage and ownership


In [277]:
df = df[(df.Year == 2019)]
df

,ISO Code,Country,Region,Year,Gender parity in account ownership,Gender gap in social media use,Gender gap in mobile ownership
5,AFG,Afghanistan,South Asia,2019,2.004152e+01,0.000000,0.000000
11,AGO,Angola,Sub-Saharan Africa,2019,5.523757e+01,49.552494,57.712311
17,ALB,Albania,Europe & Central Asia,2019,8.901575e+01,41.406120,67.741936
23,ARE,United Arab Emirates,Middle East & North Africa,2019,7.944289e+01,13.890497,100.000000
29,ARG,Argentina,Latin America & Caribbean,2019,1.000000e+02,100.000000,99.650528
...,...,...,...,...,...,...,...
995,WSM,Samoa,East Asia & Pacific,2019,9.256590e+01,100.000000,100.000000
1001,YEM,Yemen,Middle East & North Africa,2019,3.250000e-15,0.000000,0.000000
1007,ZAF,South Africa,Sub-Saharan Africa,2019,1.000000e+02,96.334473,78.272583
1013,ZMB,Zambia,Sub-Saharan Africa,2019,7.414376e+01,59.793156,53.703701


In [278]:
# create standard columns
# df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['Gender gap in mobile ownership'] 
df['Country Name'] = df['Country']
df['Sub-Pillar'] = subpillar

min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 0-1 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=0,old_max=100))

In [279]:
df = df[['Country Name', 'Year','Indicator','data_col','new_rank_score','higher_is_better','Sub-Pillar']]
df

,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar
5,Afghanistan,2019,Gender gap in mobile usage,0.000000,1.000000,True,Usage and ownership
11,Angola,2019,Gender gap in mobile usage,57.712311,3.879844,True,Usage and ownership
17,Albania,2019,Gender gap in mobile usage,67.741936,4.380323,True,Usage and ownership
23,United Arab Emirates,2019,Gender gap in mobile usage,100.000000,5.990000,True,Usage and ownership
29,Argentina,2019,Gender gap in mobile usage,99.650528,5.972561,True,Usage and ownership
...,...,...,...,...,...,...,...
995,Samoa,2019,Gender gap in mobile usage,100.000000,5.990000,True,Usage and ownership
1001,Yemen,2019,Gender gap in mobile usage,0.000000,1.000000,True,Usage and ownership
1007,South Africa,2019,Gender gap in mobile usage,78.272583,4.905802,True,Usage and ownership
1013,Zambia,2019,Gender gap in mobile usage,53.703701,3.679815,True,Usage and ownership


In [280]:
df.to_csv('../indicator_scores/people_{}_scores.csv'.format(indicator), index=False)

In [281]:
### Score Aggregating

In [282]:
import os

In [283]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('people')]

In [284]:
scores

['people_% of internet users who own cryptocurrency_scores.csv',
 'people_% of population using digital financial services_scores.csv',
 'people_% of population using Facebook_scores.csv',
 'people_% of population using internet (all)_scores.csv',
 'people_% of population using internet (female)_scores.csv',
 'people_% of population using internet (male)_scores.csv',
 'people_% of population with a smartphone_scores.csv',
 'people_Automation-led unemployment_scores.csv',
 'people_Cyberbullying_scores.csv',
 'people_E-commerce activity (% of individuals buying online and frequency)_scores.csv',
 'people_Financial Inclusiveness_scores.csv',
 'people_Gender gap for social media use_scores.csv',
 'people_Gender gap in internet usage_scores.csv',
 'people_Gender gap in mobile usage_scores.csv',
 'people_Global Crypto Adoption Index_scores.csv',
 'people_Global Wellbeing Initiative (World Happiness Index)_scores.csv',
 'people_Gross National Wellbeing_scores.csv',
 'people_Households with a 

In [285]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [286]:
df.to_csv('../pillar_scores/people_scores_beta.csv')

In [287]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df['Country Name'] = df['Country Name'].astype(str, errors = 'ignore')

# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()
df['Country Name'] = df['Country Name'].str.strip('**')
df['Country Name'] = df['Country Name'].str.strip('*')
df['Country Name'] = df['Country Name'].str.strip(' *')

# Replace values that are not truly country names with nan
df['Country Name'] = df['Country Name'].replace('nan',np.nan)

df['Country Name'] = df['Country Name'].replace('Vietnam','Viet Nam')
df['Country Name'] = df['Country Name'].replace('United States of America','United States')



# Dropping the columns having NaN/NaT values
df = df[df['Country Name'].notna()]


In [288]:
sorted(df['Country Name'].unique().tolist())

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Arab world',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bolivia (Plurinational State of)',
 'Bolivia, Plurinational State of',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Rep.',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo (Brazzaville)',
 'Congo (Democratic Republic of the)',
 'Congo (Rep. of the)',
 'Congo, Dem. Rep.',
 'Congo, Democratic Republic',
 'Congo, Rep.',
 'Congo, The Democratic Republic of the',
 'Cook Islands',
 'Costa Rica'

In [289]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [290]:
agg_df.columns = ['agg_score', 'count_source' ]

In [291]:
max_number_sources = agg_df.describe()['count_source']['max']

In [292]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [293]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [294]:
agg_df.head(25)

,agg_score,count_source,agg_score_wt
Country Name,,,
Korea (Rep. of),5.809849,8,1.659957
"Hong Kong, China (SAR)",5.735510,1,0.204840
"Hong Kong, China",5.647375,8,1.613536
United Kingdom of Great Britain and Northern Ireland,5.636708,1,0.201311
Korea (Republic of),5.570840,1,0.198959
Brunei,5.517946,1,0.197070
Republic of Korea,5.489503,1,0.196054
"Macao, China",5.367137,7,1.341784
Georgia (Country),5.349783,1,0.191064


In [295]:
agg_df.to_csv('../pillar_scores/people_scores_v0.csv')

In [296]:
### Score Aggregating by Subpillars

In [297]:
df.insert(0,'Pillar','People')
df

,Pillar,Country Name,Year,Indicator,data_col,new_rank_score,higher_is_better,Sub-Pillar,Unnamed: 0
0,People,Afghanistan,2019.0,Gender gap in mobile usage,0.0000,1.000000,True,Usage and ownership,NaN
1,People,Afghanistan,2019.0,Global Crypto Adoption Index,0.1300,1.648700,True,Usage and ownership,NaN
2,People,Afghanistan,2019.0,Mobile Device Penetration (female),NaN,0.000000,True,Usage and ownership,NaN
3,People,Afghanistan,2020.0,Human Capital Index (HCI),0.3728,2.860272,True,Digital Literacy Skills,NaN
4,People,Afghanistan,2019.0,Mobile Device Penetration,NaN,0.000000,True,Usage and ownership,NaN
...,...,...,...,...,...,...,...,...,...
3507,People,Zimbabwe,2019.0,Gender gap in mobile usage,100.0000,5.990000,True,Usage and ownership,NaN
3508,People,Zimbabwe,2019.0,UNDP Human Development Index (HDI),0.5710,3.849290,True,Culture,153.0
3509,People,Zimbabwe,2019.0,Mobile Device Penetration (female),NaN,0.000000,True,Usage and ownership,NaN
3510,People,Zimbabwe,2019.0,Global Crypto Adoption Index,0.0400,1.199600,True,Usage and ownership,NaN


In [298]:
sub_df = df.groupby(['Pillar','Sub-Pillar','Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [299]:
sub_df.columns = ['agg_score', 'count_source' ]

In [300]:
max_number_sources = sub_df.describe()['count_source']['max']

In [301]:
sub_df['agg_score_wt'] = sub_df['agg_score']*(sub_df['count_source']/max_number_sources)

In [302]:
sub_df.to_csv('../subpillar_score/people_scores_subpillar_v0.csv')

### Sources Generation

In [303]:
#Get all countries from Countries.xlsx
countries = pd.read_excel('../../data/Countries.xlsx')
col_names = ['Country or Area']
countries = countries[col_names]
countries.rename(columns = {'Country or Area': 'Country Name'}, inplace = True)

In [304]:
#Get all indicators from names dataframe retrieve at the begining of the script
bnames=bnames[['check','Sub-Pillar','Indicator','Data Source','Data Link']]
bnames.rename(columns = {'check': 'Pillar'}, inplace = True)
bnames = bnames.replace('\n','', regex=True)

D:\Apps\Anaconda\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [305]:
#Do a nice cross join so that we have combination of all countries vs all indicators
sources = countries.merge(bnames, how='cross')
sources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link
0,Algeria,People,Digital Literacy Skills,Human Capital Index (HCI),UN: E-Government Survey,https://publicadministration.un.org/egovkb
1,Algeria,People,Digital Literacy Skills,% of population using internet (all),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
2,Algeria,People,Digital Literacy Skills,% of population using internet (female),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
3,Algeria,People,Digital Literacy Skills,% of population using internet (male),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
4,Algeria,People,Digital Literacy Skills,% of internet users who own cryptocurrency,Statista,https://www.statista.com
...,...,...,...,...,...,...
8710,Wallis and Futuna Islands,People,Usage and ownership,Mobile Broadband Pricing (pre-paid),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics
8711,Wallis and Futuna Islands,People,Usage and ownership,Tax as % of total cost of mobile ownership,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com
8712,Wallis and Futuna Islands,People,Usage and ownership,% of population with a smartphone,Statista,https://www.statista.com
8713,Wallis and Futuna Islands,People,Usage and ownership,Gender gap in internet usage,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics


In [306]:
#Make copy of a scores dataframe and add the column available, with value of 1 (string)
#denoting all the country/indicator combinations that have value
dfsources = df[['Country Name','Pillar','Sub-Pillar','Indicator']].copy()
dfsources['Available'] = '1'

In [307]:
#Merge (left join) sources (all country indicator combinations)
#with those in copied scores dataframe
#resulting in a sources dataframe with values of 1 in Available column where there is a value for country/indicator
#and there is a NaN where there's not. 
#subsequently replace NaN with 0 (string)
sources = sources.merge(dfsources, how='left', on=['Country Name','Pillar','Sub-Pillar','Indicator'])
sources['Available'] = sources['Available'].fillna('0')
sources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link,Available
0,Algeria,People,Digital Literacy Skills,Human Capital Index (HCI),UN: E-Government Survey,https://publicadministration.un.org/egovkb,1
1,Algeria,People,Digital Literacy Skills,% of population using internet (all),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
2,Algeria,People,Digital Literacy Skills,% of population using internet (female),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
3,Algeria,People,Digital Literacy Skills,% of population using internet (male),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
4,Algeria,People,Digital Literacy Skills,% of internet users who own cryptocurrency,Statista,https://www.statista.com,0
...,...,...,...,...,...,...,...
8710,Wallis and Futuna Islands,People,Usage and ownership,Mobile Broadband Pricing (pre-paid),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
8711,Wallis and Futuna Islands,People,Usage and ownership,Tax as % of total cost of mobile ownership,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,0
8712,Wallis and Futuna Islands,People,Usage and ownership,% of population with a smartphone,Statista,https://www.statista.com,0
8713,Wallis and Futuna Islands,People,Usage and ownership,Gender gap in internet usage,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0


In [308]:
#If sources.csv exists, get the contents, remove everyhing from this pillar, append prepared sources, save csv.
#if sources.csv does not exist, create new file from sources.
from os.path import exists

if exists('../../dashboard/Sources.csv') :
    CurrentSources = pd.read_csv('../../dashboard/Sources.csv', dtype=str)
    CurrentSources = CurrentSources[['Country Name','Pillar','Sub-Pillar','Indicator','Data Source','Data Link','Available']]
    CurrentSources = CurrentSources.loc[CurrentSources['Pillar'] != 'People']
    CurrentSources = CurrentSources.append(sources)
else :
    CurrentSources = sources
CurrentSources

,Country Name,Pillar,Sub-Pillar,Indicator,Data Source,Data Link,Available
0,Algeria,Foundations,Digital Payments,Digital payments penetration,Portulans Institute: Network Readiness Index,https://networkreadinessindex.org,0
1,Algeria,Foundations,Digital Payments,% of population with digital finance account,World Bank: Global Findex database,https://datacatalog.worldbank.org,1
2,Algeria,Foundations,Digital Payments,Made or received digital payments in the past ...,World Bank: Global Findex database,https://datacatalog.worldbank.org,1
3,Algeria,Foundations,Digital Payments,Made or received digital payments in the past ...,World Bank: Global Findex database,https://datacatalog.worldbank.org,1
4,Algeria,Foundations,Digital Payments,Used a mobile phone or the internet to check a...,World Bank: Global Findex database,https://datacatalog.worldbank.org,1
...,...,...,...,...,...,...,...
8710,Wallis and Futuna Islands,People,Usage and ownership,Mobile Broadband Pricing (pre-paid),ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0
8711,Wallis and Futuna Islands,People,Usage and ownership,Tax as % of total cost of mobile ownership,GSMA: Mobile Connectivity Index,https://www.mobileconnectivityindex.com,0
8712,Wallis and Futuna Islands,People,Usage and ownership,% of population with a smartphone,Statista,https://www.statista.com,0
8713,Wallis and Futuna Islands,People,Usage and ownership,Gender gap in internet usage,ITU: Digital Development Dashboard,https://www.itu.int/itu-d/sites/statistics,0


In [309]:
CurrentSources.to_csv('../../dashboard/Sources.csv', index=False)